# [高效编程视频]python 中的 GIL

## GIL(global interpreter lock)全局解释器锁（cpython）

为了使得线程运行安全，尤其是多线程，python 会在解释器上加一把锁， 使得使得同一个时刻只有一个线程在一个cpu上执行字节码，无法将多个线程映射到多个CPU上执行，无法体现多CPU优势。

- python中一个线程对应c语言的一个线程

- python运行时的过程是把py文件编译成字节码

dis库是python(默认的CPython)自带的一个库,可以用来分析字节码

Python代码是先被编译为Python字节码后，再由Python虚拟机来执行Python字节码（pyc文件主要就是用于存储字节码指令 的）。一般来说一个Python语句会对应若干字节码指令，Python的字节码是一种类似汇编指令的中间语言，但是一个字节码指令并不是对应一个机器指 令（二进制指令），而是对应一段C代码，而不同的指令的性能不同，所以不能单独通过指令数量来判断代码的性能，而是要通过查看调用比较频繁的指令的代码来 确认一段程序的性能。

In [10]:
import dis

def add(a):
    a = a + 1
    return a

print(dis.dis(add))

  4           0 LOAD_FAST                0 (a)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 STORE_FAST               0 (a)

  5           8 LOAD_FAST                0 (a)
             10 RETURN_VALUE
None


有了GIL，是不是意味编写多线程的时候是安全的，不用去考虑线程间的同步呢？实际上不是的，会在某些时刻释放

In [11]:
import threading

total = 0

def add():
    global total
    for i in range(1000000):
        total += 1
    
def sub():
    global total
    for i in range(1000000):
        total -= 1
    
thread1 = threading.Thread(target=add)
thread2 = threading.Thread(target=sub)
thread1.start()
thread2.start()
thread1.join()
thread1.join()
print(total)

-357560


每次运行的结果都不一样，这表明 GIL 会释放的。

GIL 这把锁分配给某一个线程之后，并不是说这个线程执行完了之后才会释放，再交给另一个线程。
它不是整个过程的完全占有，它会在适当的时候释放，另外一个线程就可以得到运行。GIL 结合了字节码的行数/时间片划分，比如执行了100行或者1000行之后，它会释放。

- GIL 会根据执行的字节码行数以及时间片释放
- GIL在遇到IO操作时候主动释放。

正是这个特性，使得python 多线程在IO操作频繁的情况下，非常适用的。

## 多线程编程

对于io操作来说，多线程和多进程性能差别不大

多任务可以由多进程完成，也可以由一个进程内的多线程完成。进程是由若干线程组成的，一个进程至少有一个线程。由于线程是操作系统直接支持的执行单元，因此，高级语言通常都内置多线程的支持，Python也不例外，并且，Python的线程是真正的Posix Thread，而不是模拟出来的线程。Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，threading是高级模块，对_thread进行了封装。绝大多数情况下，我们只需要使用threading这个高级模块。

启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行：

In [13]:
import time,threading

def loop():
    print('thread is running...'.format(threading.current_thread().name))
    n = 0
    while n < 5:
        n = n + 1
        print('thread {} >>> {}'.format(threading.current_thread().name,n))
        time.sleep(1)
    print('thred {} ended.'.format(threading.current_thread().name))
    
print('thread {} is running...'.format(threading.current_thread().name))

t = threading.Thread(target=loop,name='LoopThread')
#t = threading.Thread(target=loop)
t.start()
t.join()
print('thread {} ended.'.format(threading.current_thread().name))

thread MainThread is running...
thread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thred LoopThread ended.
thread MainThread ended.


- 由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程。

- Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。

- 主线程实例的名字叫 MainThread，子线程的名字在创建时指定, `t = threading.Thread(target=loop,name='LoopThread'`我们用 LoopThread 命名子线程。如果不起，python 会自动给线程命名为Thread-1，Thread-2……

### 多线程编写方式一：通过 Thread 类实例化

- target 目标函数名
- args 函数参数

In [14]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started\n')
    time.sleep(2)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

get detail html started
get detail url started
finished in 0.018999814987182617 s

get detail html end
get detail url end


问题：时间消耗0.02s？

正常理解，两个线程并行，时间消耗应该是2秒,为什么是0.01 秒

解答：我们创建了2个线程，thread1和thread2， 但其实这里有3个线程，那就是主线程，除掉线程的代码，剩下的
```
print('finished in {} s'.format(time.time()-start_time))
```
是在主线程里面运行的。 【可在 print 处 debug,Debugger 里面的 Frames,可以看到它有3个线程】

所以这里是3个线程并行，thread1开始，thread2开始，主线程开始(即`print('finished in {} s'.format(time.time()-start_time))`)

这里可以看出，主线程执行完，并没有退出，而是等待 thread1和thread2执行完才退出 (非daemon的线程会阻塞主线程的退出)


#### 需求一：当主线程退出的时候，子线程kill掉。
（jupyter notebook 结果有误）

##### thread.setDaemon(True)

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(2)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
get detail html started
get detail url started
finished in 0.0 s
```

在启动线程前设置 thread.setDaemon(True)，就是设置该线程为守护线程，表示该线程是不重要的,进程退出时不需要等待这个线程执行完成。

这样做的意义在于：避免子线程无限死循环，导致退不出程序。

上述例子，主线程执行完print后就退出了，不用等待 thread1 和 thread2完成

**如果设置1个线程为守护线程，会如何？**

In [ ]:
import time,threading

def get_detail_html(url):
    print('thread1: get detail html started')
    time.sleep(2)
    print('thread1: get detail html end')
    
def get_detail_url(url):
    print('thread2: get detail url started')
    time.sleep(2)
    print('thread2: get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    #thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
thread1: get detail html started
thread2: get detail url started
finished in 0.001001596450805664 s
thread2: get detail url end
thread1: get detail html end
```

全部打印，因为 thread1 不是守护线程，主线程执行完 print 需要等待 thread1 执行完。
而 thread2 是守护线程，只有当主线程退出才会被 kill 掉。由于thread1和thread2 需要运行完成的时间是一样的，所以全部完成。


**为了说明区别，把thread2 改成 time.sleep(4)**

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(4)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    #thread1.setDaemon(True) # 守护线程
    thread2.setDaemon(True) # 守护线程
    start_time = time.time()
    thread1.start()
    thread2.start()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
thread1: get detail html started
thread2: get detail url started
finished in 0.0009961128234863281 s
thread1: get detail html end
```

因为 thread1 不是守护线程，主线程执行完 print 需要等待 thread1 执行完。
而 thread2 是守护线程，只有当主线程退出才会被 kill 掉。由于thread1 运行完需要2秒，而 thread2 运行完需要4秒。

主线程执行完 print，等待 thread1 执行完(2秒) 就可以退出，而此时 thread2 并没执行完成。


#### 需求二：等待线程完成，之后再往下执行
（jupyter notebook 结果有误）

##### thread.join()

主线程A中，创建了子线程B，并且在主线程A中调用了B.join()，那么，主线程A会在调用的地方等待，直到子线程B完成操作后，才可以接着往下执行，那么在调用这个线程时可以使用被调用线程的join方法。

1. 阻塞主进程，专注于执行多线程中的程序。

2. 多线程多join的情况下，依次执行各线程的join方法，前头一个结束了才能执行后面一个。

3. 无参数，则等待到该线程结束，才开始执行下一个线程的join。

4. 参数timeout为线程的阻塞时间，如 timeout=2 就是罩着这个线程2s 以后，就不管他了，继续执行下面的代码。

In [ ]:
import time,threading

def get_detail_html(url):
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')
    
def get_detail_url(url):
    print('get detail url started')
    time.sleep(4)
    print('get detail url end')
    
if __name__ == '__main__':
    thread1 = threading.Thread(target=get_detail_html,args=('',)) # 用 thread 类实例化
    thread2 = threading.Thread(target=get_detail_url,args=('',)) 
    start_time = time.time()
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    print('finished in {} s'.format(time.time()-start_time))

结果如下
```
get detail html started
get detail url started
get detail html end
get detail url end
finished in 4.00110387802124 s
```
- 运行时间是4s,而不是2个线程这和，说明这里是并发
- 主线程 print 是等待 thread1 和 thread2 完成后才执行

这个多线程写法还是比较复杂，代码量小的时候还可以适用。

当代码量大，内部逻辑比较复杂，这种写法是不适用的。所以 python 有另一种常用的多线程写法。

### 多线程编写方式二：继承 Thread 类

**重载 run 方法，不是 start() 方法**

In [ ]:
import time,threading

class GetDetailHtml(threading.Thread):
    
    
    def run(self):
        print('get detail html started')
        time.sleep(2)
        print('get detail html end')
    
class GetDetailUrl(threading.Thread):
    def run(self):
        print('get detail url started')
        time.sleep(4)
        print('get detail url end')
        
if __name__ == '__main__':
    thread1 = GetDetailHtml()
    thread2 = GetDetailUrl()
    
    start_time = time.time()
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    print('finished in {} s'.format(time.time()-start_time))

结果和上面一样。

## 线程间通信

### 线程间通信方式一：共享全局变量

- 函数外全局变量
- 函数参数
- 参数放到一个py文件中

In [ ]:
import time,threading

detail_url_list = []

def get_detail_html():
    # 爬取文章详情页
    global detail_url_list
    detail_url_list.pop()
    print('get detail html started')
    time.sleep(2)
    print('get detail html end')

def get_detail_url():
    # 爬取文章列表页
    print('get detail url started')
    time.sleep(4)
    for i in range(20):
        detail_url_list.append('http://threadingtest.com/{}'.format(i))
    print('get detail url end')
    
if __name__ == '__main__':
    thread_detail_url = threading.Thread(target=get_detail_url) 
    for i in range(10):
        html_thread = threading.Thread(target=get_detail_html)
        html_thread.start()
    
#     start_time = time.time()
#     thread1.start()
#     thread2.start()
#     thread1.join()
#     thread2.join()
#     print('finished in {} s'.format(time.time()-start_time))

In [ ]:
from queue import Queue
import time,threading

def get_detail_html(queue):
    # 爬取文章详情页
    while True:
        url = queue.get()
        global detail_url_list
        detail_url_list.pop()
        print('get detail html started')
        time.sleep(2)
        print('get detail html end')

def get_detail_url(queue):
    # 爬取文章列表页
    while True:
        print('get detail url started')
        time.sleep(4)
        for i in range(20):
            queue.put('http://threadingtest.com/{}'.format(i))
        print('get detail url end')
    
if __name__ == '__main__':
    detail_url_queue = Queue(maxsize =1000)
    thread_detail_url = threading.Thread(target=get_detail_url,args=(detail_url_queue,)) 
    for i in range(10):
        html_thread = threading.Thread(target=get_detail_html,args=(detail_url_queue,))
        html_thread.start()

## 线程同步 Lock

**多线程和多进程最大的不同在于**

- 多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响。

- 多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改。

因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

来看看多个线程同时操作一个变量怎么把内容给改乱了：

In [18]:
import time, threading
import dis

# 假定这是你的银行存款
balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        change_it(n)
        
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

print(dis.dis(change_it))

12
 10           0 LOAD_GLOBAL              0 (balance)
              2 LOAD_FAST                0 (n)
              4 BINARY_ADD
              6 STORE_GLOBAL             0 (balance)

 11           8 LOAD_GLOBAL              0 (balance)
             10 LOAD_FAST                0 (n)
             12 BINARY_SUBTRACT
             14 STORE_GLOBAL             0 (balance)
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None


由于以下两个特性

- 为了使得线程运行安全，尤其是多线程，python 会在解释器上加一把锁 GIL， 使得使得同一个时刻只有一个线程在一个cpu上执行字节码

- GIL 这把锁分配给某一个线程之后，并不是说这个线程执行完了之后才会释放，再交给另一个线程。 它不是整个过程的完全占有，它会在适当的时候释放，另外一个线程就可以得到运行。

线程 t1、t2 是交替执行的。


我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

原因是因为高级语言的一条语句在CPU执行时是若干条语句，即使一个简单的计算：(反编译字节码也可以看出)

`balance = balance + n`

也分两步：

1. 计算balance + n，存入临时变量中；
2. 将临时变量的值赋给balance。

也就是可以看成：

```
x = balance + n
balance = n
```
由于x是局部变量，两个线程各自都有自己的x，当代码正常执行时：

```
t1: x1 = balance + 5 # x1 = 0 + 5 = 5
t1: balance = x1     # balance = 5
t1: x1 = balance - 5 # x1 = 5 - 5 = 0
t1: balance = x1     # balance = 0

t2: x2 = balance + 8 # x2 = 0 + 8 = 8
t2: balance = x2     # balance = 8
t2: x2 = balance - 8 # x2 = 8 - 8 = 0
t2: balance = x2     # balance = 0

结果 balance = 0
```
但是t1和t2是交替运行的，如果操作系统以下面的顺序执行t1、t2：

```
初始值 balance = 0

t1: x1 = balance + 5  # x1 = 0 + 5 = 5

t2: x2 = balance + 8  # x2 = 0 + 8 = 8
t2: balance = x2      # balance = 8

t1: balance = x1      # balance = 5
t1: x1 = balance - 5  # x1 = 5 - 5 = 0
t1: balance = x1      # balance = 0

t2: x2 = balance - 8  # x2 = 0 - 8 = -8
t2: balance = x2   # balance = -8

结果 balance = -8
```

究其原因，是因为修改balance需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。

两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：

In [ ]:
import time, threading

# 假定这是你的银行存款
balance = 0
lock = threading.Lock()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        # 先获取锁
        lock.acquire()
        try:
            change_it(n)
        finally:
            # 改完了一定要释放锁
            lock.release()
            
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

获得锁的线程用完后一定要释放锁，否则那些苦苦等待锁的线程将永远等待下去，成为死线程。所以我们用try...finally来确保锁一定会被释放。

锁的好处就是确保了某段关键代码只能由一个线程从头到尾完整地执行，坏处当然也很多，首先是阻止了多线程并发执行，包含锁的某段代码实际上只能以单线程模式执行，效率就大大地下降了。其次，由于可以存在多个锁，不同的线程持有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，导致多个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强制终止。



# Python并发编程

## [从性能角度来初探并发编程](https://zhuanlan.zhihu.com/p/36608777)

对于并发编程，Python的实现，总结了一下，大致有如下三种方法：

- 多线程
- 多进程
- 协程（生成器）

### 并发编程的基本概念


并发和并行的区别就是一个处理器同时处理多个任务和多个处理器或者是多核的处理器同时处理多个不同的任务。

前者是逻辑上的同时发生（simultaneous），而后者是物理上的同时发生。

来个比喻：并发和并行的区别就是一个人同时吃三个馒头和三个人同时吃三个馒头。

![concurrence_parallelism](concurrence_parallelism.png)

## [创建多线程的几种方法](https://zhuanlan.zhihu.com/p/36765530)

Python创建多线程主要有如下两种方法：

- 函数
- 类

### 使用函数创建多线程

在Python3中，Python提供了一个内置模块 threading.Thread，可以很方便地让我们创建多线程。

threading.Thread() 一般接收两个参数：

- 线程函数名：要放置线程让其后台执行的函数，由我们自已定义，注意不要加()；
- 线程函数的参数：线程函数名所需的参数，以元组的形式传入。若不需要参数，可以不指定。


In [6]:
import time
from threading import Thread

def main(name = 'Jamie'):
    print('hello',name)
    time.sleep(4)
    print('wait for 4 seconds to print')

# 创建线程01
thread_01 = Thread(target=main)
# 创建线程02，指定参数，注意元祖形式
thread_02 = Thread(target=main,args=('python',))

# 启动线程
thread_01.start()
thread_02.start()

hello Jamie
hello python
wait for 4 seconds to print
wait for 4 seconds to print


### 使用类创建多线程

相比较函数而言，使用类创建线程，会比较麻烦一点。

首先，我们要自定义一个类，对于这个类有两点要求，

- 必须继承 threading.Thread 这个父类；
- 必须覆写 run 方法。

这里的 run 方法，和我们上面线程函数的性质是一样的，可以写我们的业务逻辑程序。在 start() 后将会调用。

来看一下例子,为了方便对比，run函数复用上面的main。


在类的继承中，子类修改初始化方法，同时拥有父类的属性，但不覆盖父类属性。为了同时拥有父类的属性，super 的一个最常见用法可以说是在子类中调用父类的初始化方法了 `super().__init__()`。

In [8]:
import time
from threading import Thread

class MyThread(Thread):
    def __init__(self,name='Jamie'):
        super().__init__()  # 子类修改初始化方法，同时拥有父类的属性
        self.name = name

    def run(self):
        print('hello',self.name)
        time.sleep(4)
        print('wait for 4 seconds to print')    


thread_01 = MyThread()
thread_02 = MyThread('Python')

thread_01.start()
thread_02.start()

hello Jamie
hello Python
wait for 4 seconds to print
wait for 4 seconds to print


### 多线程必学函数

学完了两种创建线程的方式，你一定会惊叹，咋么这么简单，一点难度都没有。

其实不然，上面我们的线程函数 为了方便理解，都使用的最简单的代码逻辑。而在实际使用当中，多线程运行期间，还会出现诸多问题，只是我们现在还没体会到它的复杂而已。

不过，你也不必担心，在后面的章节中，我会带着大家一起来探讨一下，都有哪些难题，应该如何解决。

磨刀不误吹柴工，我们首先得来认识一下，Python给我们提供的 Thread 都有哪些函数和属性，实现哪些功能。学习完这些，在后期的学习中，我们才能更加得以应手。

经过我的总结，大约常用的方法有如下这些：

- 函数创建线程

`t = Thread(target=func)`

- 启动子线程

`t.start()`

- 堵塞子线程，待子线程结束后，再往下执行

`t.join()`

- 判断子线程是否在执行状态，在执行返回 True,否则返回 False

`t.is_alive()` or `t.isAlive()`

- 设置线程是否随主线程退出而退出，默认为 False

`t.daemon = True` or `t.daemon = False`

- 设置线程名
`t.name = 'My_thread'`

## [谈谈线程中的“锁机制”](https://zhuanlan.zhihu.com/p/36766159)

加锁是为了对锁内资源（变量）进行锁定，避免其他线程篡改已被锁定的资源，以达到我们预期的效果。

### 如何使用Lock（ 锁 ）

```
import threading

lock = threading.lock() # 生成锁对象，全局唯一
lock.acquire()  # 获取锁。未获取到会堵塞程序，直到获取到锁才会wj
lock.release() # 释放锁，释放锁，其他线程可以拿去用了
```

需要注意的是，lock.acquire() 和 lock.release()必须成对出现。否则就有可能造成死锁。

### 上下文管理器来加锁

很多时候，我们虽然知道，他们必须成对出现，但是还是难免会有忘记的时候。为了，规避这个问题。我推荐使用使用上下文管理器来加锁。

```
import threading

lock = threading.lock()
with lock:
    # 这里写自己的代码
    pass
```

**with 语句会在这个代码块执行前自动获取锁，在执行结束后自动释放锁。**

### 可重入锁（RLock）

有时候在同一个线程中，我们可能会多次请求同一资源，俗称锁嵌套。

如果还是按照常规的做法，会造成死锁的。比如，下面这段代码，你可以试着运行一下。会发现并没有输出结果。

In [2]:
import threading

def main():
    n = 0
    lock = threading.Lock()
    with lock:
        for i in range(10):
            n += 1
            with lock:
                print(n)

t1 = threading.Thread(target=main)
t1.start()

是因为，第二次获取锁时，发现锁已经被同一线程的人拿走了。自己也就理所当然，拿不到锁，程序就卡住了。

那么如何解决这个问题呢。

threading模块除了提供Lock锁之外，还提供了一种可重入锁RLock，专门来处理这个问题。

In [3]:
import threading

def main():
    n = 0
     # 生成可重入锁对象
    lock = threading.RLock()
    with lock:
        for i in range(10):
            n += 1
            with lock:
                print(n)

t1 = threading.Thread(target=main)
t1.start()

1
2
3
4
5
6
7
8
9
10


发现已经有输出了。 需要注意的是，可重入锁，只在同一线程里，放松对锁的获取机制，允许同一线程里的多次对锁进行获取，其他与Lock并无二致。

### Lock 和 RLock 的区别
https://blog.csdn.net/davidsu33/article/details/51385965

**Lock**

In [ ]:
import threading  
lock = threading.Lock() #Lock对象  
lock.acquire()  
lock.acquire()  #产生了死琐。  
lock.release()  
lock.release() 

**Rlock**

In [ ]:
import threading  
rLock = threading.RLock()  #RLock对象  
rLock.acquire()  
rLock.acquire() #在同一线程内，程序不会堵塞。  
rLock.release()  
rLock.release()  

这两种琐的主要区别是：**RLock允许在同一线程中被多次acquire。而Lock却不允许这种情况。**

注意：如果使用RLock，那么acquire和release必须成对出现，即调用了n次acquire，必须调用n次的release才能真正释放所占用的琐。

当需要在类外面保证线程安全，又要在类内使用同样方法的时候RLock()就很适用。

RLock叫做Reentrant Lock，就是可以重复进入的锁，也叫递归锁。这种锁对比Lock有三个特点：1、谁拿到锁，谁释放；2、同一线程可以多次拿到该锁；3、acquire多少次就必须release多少次，只有最后一次release才能改变RLock的状态为unlocked。

### 防止死锁的加锁机制 (没懂)


### 饱受争议的GIL（全局锁）

多进程是真正的并行，而多线程是伪并行，实际上他只是交替执行。

是什么导致多线程，只能交替执行呢？是一个叫GIL（Global Interpreter Lock，全局解释器锁）的东西。

什么是GIL呢？

任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。

需要注意的是，GIL并不是Python的特性，它是在实现Python解析器(CPython)时所引入的一个概念。而Python解释器，并不是只有CPython，除它之外，还有PyPy，Psyco，JPython，IronPython等。

在绝大多数情况下，我们通常都认为 Python == CPython，所以也就默许了Python具有GIL锁这个事。

都知道GIL影响性能，那么如何避免受到GIL的影响？

- 使用多进程代替多线程。

- 更换Python解释器，不使用CPython

## [线程消息通信机制/任务协调](https://zhuanlan.zhihu.com/p/36766159)

前面我已经向大家介绍了，如何使用创建线程，启动线程。相信大家都会有这样一个想法，线程无非就是创建一下，然后再start()下，实在是太简单了。

可是要知道，在真实的项目中，实际场景可要我们举的例子要复杂的多得多，不同线程的执行可能是有顺序的，或者说他们的执行是有条件的，是要受控制的。如果仅仅依靠前面学的那点浅薄的知识，是远远不够的。

那今天，我们就来探讨一下如何控制线程的触发执行。

要实现对多个线程进行控制，其实本质上就是消息通信机制在起作用，利用这个机制发送指令，告诉线程，什么时候可以执行，什么时候不可以执行，执行什么内容。

经过我的总结，线程中通信方法大致有如下三种：

- threading.Event

- threading.Condition

- queue.Queue

先抛出结论，接下来我们来一一探讨下。

### Event事件

Python提供了非常简单的通信机制 Threading.Event，通用的条件变量。多个线程可以等待某个事件的发生，在事件发生后，所有的线程都会被激活。

条件同步和条件变量（condition）同步差不多意思，只是少了锁功能，因为条件同步设计于不访问共享资源的条件环境。

用于线程间通信，即程序中的其一个线程需要通过判断某个线程的状态来确定自己下一步的操作，就用到了event对象。 event对象默认为假（Flase），即遇到event对象在等待就阻塞线程的执行。

![event](event.png)


关于Event的使用也超级简单，就三个函数

- 重置event，使得所有该event事件都处于待命状态
`event.clear()`
- 等待接收event的指令，决定是否阻塞程序执行
`event.wait()`
- 发送event指令，使所有设置该event事件的线程执行
`event.set()`

举个例子来看下。

In [7]:
import time
import threading

class MyThread(threading.Thread):
    def __init__(self,name,event):
        super().__init__()
        self.name = name
        self.event = event
        
    def run(self):
        print('Thread:{} start at {}'.format(self.name,time.ctime(time.time())))
         # 等待event.set()后，才能往下执行
        self.event.wait()
        print('Thread:{} finish at {}'.format(self.name,time.ctime(time.time())))
        

event = threading.Event()

# 定义五个线程
threads = [MyThread(i,event)for i in range(1,5)]

# 重置event，使得event.wait()起到阻塞作用
event.clear()

# 启动所有线程
for t in threads:
    t.start()

print('等待5s...')
time.sleep(5)
print('唤醒所有线程...')
event.set()

Thread:1 start at Thu Apr 11 12:07:27 2019Thread:2 start at Thu Apr 11 12:07:27 2019

Thread:3 start at Thu Apr 11 12:07:27 2019Thread:4 start at Thu Apr 11 12:07:27 2019等待5s...


唤醒所有线程...
Thread:3 finish at Thu Apr 11 12:07:32 2019
Thread:2 finish at Thu Apr 11 12:07:32 2019
Thread:4 finish at Thu Apr 11 12:07:32 2019
Thread:1 finish at Thu Apr 11 12:07:32 2019


可见在所有线程都启动（start()）后，并不会执行完，而是都在self.event.wait()止住了，需要我们通过event.set()来给所有线程发送执行指令才能往下执行。

### Condition

Condition和Event 是类似的，并没有多大区别。

同样，Condition也只需要掌握几个函数即可。

`cond = threading.Condition()`

- 类似lock.acquire()
`cond.acquire()`

- 类似lock.release()
`cond.release()`

- 等待指定触发，同时会释放对锁的获取,直到被notify才重新占有琐。
`cond.wait()`

- 发送指定，触发执行
`cond.notify()`

举个网上一个比较趣的捉迷藏的例子来看看

In [8]:
import time,threading

class Hider(threading.Thread):
    def __init__(self,cond,name):
        super().__init__()
        self.cond = cond
        self.name = name
        
    def run(self):
        time.sleep(1) # 确保先运行 Seeker 中的方法
        self.cond.acquire()
        print('{}:我已经把眼镜蒙上了'.format(self.name))
        self.cond.notify()
        self.cond.wait()
        print('{}:我找到你了哦'.format(self.name))
        self.cond.notify()
        self.cond.release()
        print('{}:我赢了'.format(self.name))

class Seeker(threading.Thread):
    def __init__(self,cond,name):
        super().__init__()
        self.cond = cond
        self.name = name
        
    def run(self):
        self.cond.acquire()
        self.cond.wait()
        print('{}:我已经藏好了，你快来找我吧'.format(self.name))
        self.cond.notify()
        self.cond.wait()
        self.cond.notify()
        self.cond.release()
        print('{}:被你找到了，哎'.format(self.name))  
        
cond = threading.Condition()
seeker = Seeker(cond,'seeker')
hider = Hider(cond,'hider')
seeker.start()
hider.start()

hider:我已经把眼镜蒙上了
seeker:我已经藏好了，你快来找我吧
hider:我找到你了哦
hider:我赢了
seeker:被你找到了，哎


### Queue队列

从一个线程向另一个线程发送数据最安全的方式可能就是使用 queue 库中的队列了。创建一个被多个线程共享的 Queue 对象，这些线程通过使用put() 和 get() 操作来向队列中添加或者删除元素。

同样，对于Queue，我们也只需要掌握几个函数即可。

```
from queue import Queue
# maxsize默认为0，不受限
# 一旦>0，而消息数又达到限制，q.put()也将阻塞
q = Queue(maxsize=0)

# 阻塞程序，等待队列消息。
q.get()

# 获取消息，设置超时时间
q.get(timeout=5.0)

# 发送消息
q.put()

# 等待所有的消息都被消费完
q.join()


# 查询当前队列的消息个数
q.qsize()

# 队列消息是否都被消费完，True/False
q.empty()

# 检测队列里消息是否已满
q.full()
```

In [5]:
from queue import Queue
from threading import Thread
import time

class Student(Thread):
    def __init__(self, name, queue):
        super().__init__()
        self.name = name
        self.queue = queue

    def run(self):
        while True:
            # 阻塞程序，时刻监听老师，接收消息
            msg = self.queue.get()
            # 一旦发现点到自己名字，就赶紧答到
            if msg == self.name:
                print("{}：到！".format(self.name))
                

class Teacher:
    def __init__(self, queue):
        self.queue=queue

    def call(self, student_name):
        print("老师：{}来了没？".format(student_name))
        # 发送消息，要点谁的名
        self.queue.put(student_name)

queue = Queue()
teacher = Teacher(queue=queue)
s1 = Student(name="小明", queue=queue)
s2 = Student(name="小亮", queue=queue)
s1.start()
s2.start()

print('开始点名~')
teacher.call('小明')
time.sleep(1)
teacher.call('小亮')
time.sleep(1)

开始点名~
老师：小明来了没？
小明：到！
老师：小亮来了没？
小亮：到！


#### 总结

学习了以上三种通信方法，我们很容易就能发现Event 和 Condition 是threading模块原生提供的模块，原理简单，功能单一，它能发送 True 和 False 的指令，所以只能适用于某些简单的场景中。

而Queue则是比较高级的模块，它可能发送任何类型的消息，包括字符串、字典等。其内部实现其实也引用了Condition模块（譬如put和get函数的阻塞），正是其对Condition进行了功能扩展，所以功能更加丰富，更能满足实际应用。

# [Python 多线程](http://python.jobbole.com/85177/)

## 线程状态

创建线程之后，线程并不是始终保持一个状态。其状态大概如下：

- New 创建。
- Runnable 就绪。等待调度
- Running 运行。
- Blocked 阻塞。阻塞可能在 Wait Locked Sleeping
- Dead 消亡

这些状态之间是可以相互转换的：

![threading_state](threading_state.jpg)

线程中执行到阻塞，可能有3种情况：

- 同步：线程中获取同步锁，但是资源已经被其他线程锁定时，进入Locked状态，直到该资源可获取（获取的顺序由Lock队列控制）
- 睡眠：线程运行sleep()或join()方法后，线程进入Sleeping状态。区别在于sleep等待固定的时间，而join是等待子线程执行完。当然join也可以指定一个“超时时间”。从语义上来说，如果两个线程a,b, 在a中调用b.join()，相当于合并(join)成一个线程。最常见的情况是在主线程中join所有的子线程。
- 等待：线程中执行wait()方法后，线程进入Waiting状态，等待其他线程的通知(notify）。

## 线程类型

线程有着不同的状态，也有不同的类型。大致可分为：

- 主线程
- 子线程
- 守护线程（后台线程）
- 前台线程

## Python线程与GIL

**相比进程，线程更加轻量，可以实现并发**。可是在python的世界里，对于线程，就不得不说一句GIL(全局解释器锁)。GIL的存在让python的多线程多少有点鸡肋了。Cpython的线程是操作系统原生的线程在解释器解释执行任何Python代码时，都需要先获得这把锁才行，在遇到 I/O 操作时会释放这把锁。因为python的进程做为一个整体，解释器进程内只有一个线程在执行，其它的线程都处于等待状态等着GIL的释放。

关于GIL可以有更多的趣事，一时半会都说不完。总之python想用多线程并发，效果可能还不如单线程（线程切换耗时间）。想要利用多核，可以考虑使用多进程。

## 线程的创建

虽然python线程比较鸡肋，可是也并发一无是处。多了解还是有理由对并发模型的理解。

Python提供两个模块进行多线程的操作，分别是thread和threading，前者是比较低级的模块，用于更底层的操作，一般应有级别的开发不常用。后者则封装了更多高级的接口，类似java的多线程风格，提供run方法和start调用。

In [21]:
import time
import threading

class MyThread(threading.Thread):
    def run(self):
        for i in range(3):
            print('thread {}, @number:{},time{}'.format(self.name,i,time.time()))
            time.sleep(1)
            
def main():
    print('Start main threaing,time:{}'.format(time.time()))
    # 创建三个线程
    threads = [MyThread() for i in range(3)]
    # 启动三个线程
    for t in threads:
        t.start()
    print('End main threading,time:{}'.format(time.time()))
    
if __name__ == '__main__':
    start_time = time.time()
    main()
    print('finished in {} s'.format(time.time()-start_time))

Start main threaing,time:1554967328.7502546
thread Thread-52, @number:0,time1554967328.755258thread Thread-53, @number:0,time1554967328.7642548

thread Thread-54, @number:0,time1554967328.7722538End main threading,time:1554967328.7802613

finished in 0.03099799156188965 s
thread Thread-52, @number:1,time1554967329.7733057thread Thread-53, @number:1,time1554967329.7733057

thread Thread-54, @number:1,time1554967329.782286
thread Thread-53, @number:2,time1554967330.7742543
thread Thread-52, @number:2,time1554967330.775258
thread Thread-54, @number:2,time1554967330.7833207


每个线程都依次打印 0 – 3 三个数字，可是从输出的结果观察，线程并不是顺序的执行，而是三个线程之间相互交替执行。此外，我们的主线程执行结束，将会打印 End Main threading。从输出结果可以知道，主线程结束后，新建的线程还在运行。

## 线程合并（join方法）

上述的例子中，主线程结束了，子线程还在运行。如果需要主线程等待子线程执行完毕再退出，可是使用线程的join方法。join方法官网文档大概是

>join(timeout)方法将会等待直到线程结束。这将阻塞正在调用的线程，直到被调用join()方法的线程结束。

主线程或者某个函数如果创建了子线程，只要调用了子线程的join方法，那么主线程就会被子线程所阻塞，直到子线程执行完毕再轮到主线程执行。其结果就是所有子线程执行完毕，才打印 End Main threading。只需要修改上面的main函数

In [18]:
import time
import threading

class MyThread(threading.Thread):
    def run(self):
        for i in range(3):
            print('thread {}, @number:{},time{}'.format(self.name,i,time.time()))
            time.sleep(1)
            
def main():
    print('Start main threaing,time:{}'.format(time.time()))
    # 创建三个线程
    threads = [MyThread() for i in range(3)]
    # 启动三个线程
    for t in threads:
        t.start()
    # 一次让新创建的线程执行 join
    for t in threads:
        t.join()
    print('End main threading,time:{}'.format(time.time()))
    
if __name__ == '__main__':
    start_time = time.time()
    main()
    print('finished in {} s'.format(time.time()-start_time))

Start main threaing,time:1554967267.844993
thread Thread-43, @number:0,time1554967267.8469758
thread Thread-44, @number:0,time1554967267.8579717
thread Thread-45, @number:0,time1554967267.8669744
thread Thread-43, @number:1,time1554967268.8589747
thread Thread-44, @number:1,time1554967268.8670413
thread Thread-45, @number:1,time1554967268.873979
thread Thread-43, @number:2,time1554967269.8599885
thread Thread-44, @number:2,time1554967269.868093
thread Thread-45, @number:2,time1554967269.8750844
End main threading,time:1554967270.8760836
finished in 3.031090497970581 s


所有子线程结束了才会执行也行print "End Main threading"。其实join很好理解，就字面上的意思就是子线程 “加入”（join）主线程嘛。在CPU执行时间片段上“等于”主线程的一部分。

如果在 t.start()之后join会怎么样？结果也能阻塞主线程，但是每个线程都是依次执行，变得有顺序了。在start之后join，也就是每个子线程由被后来新建的子线程给阻塞了，因此线程之间变得有顺序了。如下，效果就等同于没有使用多线程

In [20]:
import time
import threading

class MyThread(threading.Thread):
    def run(self):
        for i in range(3):
            print('thread {}, @number:{},time{}'.format(self.name,i,time.time()))
            time.sleep(1)
            
def main():
    print('Start main threaing,time:{}'.format(time.time()))
    # 创建三个线程
    threads = [MyThread() for i in range(3)]
    # 启动三个线程
    for t in threads:
        t.start()
        t.join()
    print('End main threading,time:{}'.format(time.time()))
    
if __name__ == '__main__':
    start_time = time.time()
    main()
    print('finished in {} s'.format(time.time()-start_time))

Start main threaing,time:1554967298.2052548
thread Thread-49, @number:0,time1554967298.212255
thread Thread-49, @number:1,time1554967299.2233164
thread Thread-49, @number:2,time1554967300.225251
thread Thread-50, @number:0,time1554967301.22727
thread Thread-50, @number:1,time1554967302.2483456
thread Thread-50, @number:2,time1554967303.249317
thread Thread-51, @number:0,time1554967304.250337
thread Thread-51, @number:1,time1554967305.257263
thread Thread-51, @number:2,time1554967306.258339
End main threading,time:1554967307.259251
finished in 9.053996324539185 s


**join方法总结**
1. join方法的作用是阻塞主进程（挡住，无法执行join以后的语句），专注执行多线程。
2. 多线程多join的情况下，依次执行各线程的join方法，前头一个结束了才能执行后面一个。
3. 无参数，则等待到该线程结束，才开始执行下一个线程的join。
4. 设置参数后，则等待该线程这么长时间就不管它了（而该线程并没有结束）。不管的意思就是可以执行后面的主进程了

## 线程同步与互斥锁

**线程之所以比进程轻量，其中一个原因就是他们共享内存**。也就是各个线程可以平等的访问内存的数据，如果在短时间“同时并行”读取修改内存的数据，很可能造成数据不同步。例如下面的例子：

In [29]:
import time, threading
import dis

# 假定这是你的银行存款
balance = 0

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        change_it(n)
        
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

print(dis.dis(change_it))

10
 10           0 LOAD_GLOBAL              0 (balance)
              2 LOAD_FAST                0 (n)
              4 BINARY_ADD
              6 STORE_GLOBAL             0 (balance)

 11           8 LOAD_GLOBAL              0 (balance)
             10 LOAD_FAST                0 (n)
             12 BINARY_SUBTRACT
             14 STORE_GLOBAL             0 (balance)
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None


由于以下两个特性

- 为了使得线程运行安全，尤其是多线程，python 会在解释器上加一把锁 GIL， 使得使得同一个时刻只有一个线程在一个cpu上执行字节码

- GIL 这把锁分配给某一个线程之后，并不是说这个线程执行完了之后才会释放，再交给另一个线程。 它不是整个过程的完全占有，它会在适当的时候释放，另外一个线程就可以得到运行。

线程 t1、t2 是交替执行的。


我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

原因是因为高级语言的一条语句在CPU执行时是若干条语句，即使一个简单的计算：(反编译字节码也可以看出)

`balance = balance + n`

也分两步：

1. 计算balance + n，存入临时变量中；
2. 将临时变量的值赋给balance。

也就是可以看成：

```
x = balance + n
balance = n
```
由于x是局部变量，两个线程各自都有自己的x，当代码正常执行时：

```
t1: x1 = balance + 5 # x1 = 0 + 5 = 5
t1: balance = x1     # balance = 5
t1: x1 = balance - 5 # x1 = 5 - 5 = 0
t1: balance = x1     # balance = 0

t2: x2 = balance + 8 # x2 = 0 + 8 = 8
t2: balance = x2     # balance = 8
t2: x2 = balance - 8 # x2 = 8 - 8 = 0
t2: balance = x2     # balance = 0

结果 balance = 0
```
但是t1和t2是交替运行的，如果操作系统以下面的顺序执行t1、t2：

```
初始值 balance = 0

t1: x1 = balance + 5  # x1 = 0 + 5 = 5

t2: x2 = balance + 8  # x2 = 0 + 8 = 8
t2: balance = x2      # balance = 8

t1: balance = x1      # balance = 5
t1: x1 = balance - 5  # x1 = 5 - 5 = 0
t1: balance = x1      # balance = 0

t2: x2 = balance - 8  # x2 = 0 - 8 = -8
t2: balance = x2   # balance = -8

结果 balance = -8
```

究其原因，是因为修改balance需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。

两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：

In [30]:
import time, threading

# 假定这是你的银行存款
balance = 0
lock = threading.Lock()

def change_it(n):
    # 先存后取，结果应该为0:
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000):
        # 先获取锁
        lock.acquire()
        try:
            change_it(n)
        finally:
            # 改完了一定要释放锁
            lock.release()
            
t1 = threading.Thread(target=run_thread, args=(5,))
t2 = threading.Thread(target=run_thread, args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

获得锁的线程用完后一定要释放锁，否则那些苦苦等待锁的线程将永远等待下去，成为死线程。所以我们用try...finally来确保锁一定会被释放。

锁的好处就是确保了某段关键代码只能由一个线程从头到尾完整地执行，坏处当然也很多，首先是阻止了多线程并发执行，包含锁的某段代码实际上只能以单线程模式执行，效率就大大地下降了。其次，由于可以存在多个锁，不同的线程持有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，导致多个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强制终止。

## 死锁

有锁就可以方便处理线程同步问题，可是多线程的复杂度和难以调试的根源也来自于线程的锁。利用不当，甚至会带来更多问题。比如死锁就是需要避免的问题。

In [40]:
lock_a = threading.Lock()
lock_b = threading.Lock()
 
class MyThread(threading.Thread):
    def task_a(self):
        if lock_a.acquire(): # 获取到 lock_a
            print ("thread {} get lock a ".format(self.name))
            time.sleep(1)
            if lock_b.acquire(): # lock_b被 task_b 获取
                print ("thread {} get lock b ".format(self.name))
                lock_b.release()
            lock_a.release()
 
    def task_b(self):
        if lock_b.acquire(): # 获取到 lock_b
            print ("thread {} get lock a ".format(self.name))
            time.sleep(1)
            if lock_a.acquire(): # lock_a 被 task_a 获取
                print ("thread {} get lock b ".format(self.name))
                lock_a.release()
            lock_b.release()
            
    def run(self):
        self.task_a()
        self.task_b()

def main():
    print ("Start main threading")
    threads = [MyThread() for i in range(2)]
    for t in threads:
        t.start()
    print ("End Main threading")

线程需要执行两个任务，两个任务都需要获取锁，然而两个任务先得到锁后，就需要等另外锁释放。

## 可重入锁(同一线程中)

为了支持在**同一线程中**多次请求同一资源(既连续调用 require)python提供了可重入锁（RLock）。RLock内部维护着一个Lock和一个counter变量，counter记录了acquire的次数，从而使得资源可以被多次require。直到一个线程所有的acquire都被release，其他的线程才能获得资源。

In [42]:
lock = threading.RLock()
 
class MyThread(threading.Thread):
    def run(self):
        if lock.acquire(1):
            print ("thread {} get lock".format(self.name))
            time.sleep(1)
            lock.acquire()
            lock.release()
            lock.release()

def main():
    print ("Start main threading")
    threads = [MyThread() for i in range(2)]
    for t in threads:
        t.start()
    print("End Main threading")
    
main()

Start main threading
thread Thread-211 get lock
End Main threading
thread Thread-212 get lock


## 条件变量(用于复杂的线程间同步)

实用锁可以达到线程同步，前面的互斥锁就是这种机制。更复杂的环境，需要针对锁进行一些条件判断。Python提供了Condition对象。它除了具有acquire和release方法之外，还提供了wait和notify方法。线程首先acquire一个条件变量锁。如果条件不足，则该线程wait，如果满足就执行线程，甚至可以notify其他线程。其他处于wait状态的线程接到通知后会重新判断条件。


Condition对象有四个最常用的主要方法

1. Condition对象 . acquire()
此方法直接传递给Condition对象的Lock对象使用，与Lock对象的作用一样，用于获取锁，并让并行的其它线程处于不执行状态，让当前线程独占运行。

2. Condition对象 . release()
此方法也直接传递给Condition对象的Lock对象使用，与Lock对象的作用一样，用于释放对锁的锁定。

3. Condition对象 . notify()
必须在acquire方法执行之后才能执行此方法，否则就会报错。执行此方法后，会通知线程池中处于wait状态的其中一个线程，让它尝试获取锁并继续运行。注意此方法并不自动释放锁，因此执行完此方法后，一般应当马上执行release方法。

4. Condition对象 . wait()
必须在acquire方法执行之后才能执行此方法，否则就会报错。执行此方法后，会将当前线程放入wait线程池中，也就是让当前线程处于wait状态。注意此方法并不自动释放锁，因此执行完此方法后，一般应当马上执行release方法。


条件变量可以看成不同的线程先后acquire获得锁，如果不满足条件，可以理解为被扔到一个（Lock或RLock）的waiting池。直达其他线程notify之后再重新判断条件。

### [python 线程间同步之条件变量Condition](https://www.jianshu.com/p/5d2579938517)

先看一个互斥锁解决不了的场景，假设两个智能聊天机器人（小米的小爱和天猫的天猫精灵）对话

In [1]:
import threading

class XiaoAi(threading.Thread):
    def __init__(self, lock):
        super().__init__(name='小爱') # threading.Thread类中有 name 属性
        self.lock = lock
        
    def run(self):
        self.lock.acquire()
        print("{} : 在".format(self.name))
        self.lock.release()
        self.lock.acquire()
        print("{} : 好啊".format(self.name))
        self.lock.release()
        
class TianMao(threading.Thread):
    def __init__(self, lock):
        super().__init__(name='天猫精灵')
        self.lock = lock
        
    def run(self):
        self.lock.acquire()
        print("{} : 小爱同学".format(self.name))
        self.lock.release()
        self.lock.acquire()
        print("{} :我们来对古诗吧".format(self.name))
        self.lock.release() 
        
if __name__ == "__main__":
    lock = threading.Lock()
    xiaoai = XiaoAi(lock)
    tianmao = TianMao(lock)
    tianmao.start()
    xiaoai.start()

天猫精灵 : 小爱同学
天猫精灵 :我们来对古诗吧
小爱 : 在
小爱 : 好啊


可以看到，输出结果并不是预期的对话顺序，这是因为天猫精灵的线程说完“小爱同学”之后，cpu的控制权还没有交出去，继续获取了互斥锁，又执行了“我们来对古诗吧”，所以不能得到预期结果。

先自己想一下解决办法，理论上应该A线程在等待中，B线程在干活，干活完毕之后通知A线程活干完了，B线程进入等待，而A线程得到了通知之后，不再继续等待，开始干活，看完之后通知B线程，如此循环，直到结束。

`with self.cond`

等同于

```
self.cond.require()
self.cond.release()
```

In [39]:
import threading,time

class XiaoAi(threading.Thread):
    def __init__(self, cond):
        super().__init__(name='小爱') # threading.Thread类中有 name 属性
        self.cond = cond
        
    def run(self):
        with self.cond: 
            self.cond.wait() 
            print("{} : 在".format(self.name))
            self.cond.notify()  
            self.cond.wait()
            print("{} : 好啊".format(self.name))
            self.cond.notify() 


class TianMao(threading.Thread):
    def __init__(self, cond):
        super().__init__(name='天猫精灵')
        self.cond = cond
        
    def run(self):
        with self.cond: 
            print("{} : 小爱同学".format(self.name))
            self.cond.notify() 
            self.cond.wait() 
            print("{} :我们来对古诗吧".format(self.name))
            self.cond.notify() 
            self.cond.wait() 
            

if __name__ == "__main__":
    con = threading.Condition()
    xiaoai = XiaoAi(con)
    tianmao = TianMao(con)
    xiaoai.start()
    tianmao.start()

天猫精灵 : 小爱同学
小爱 : 在
天猫精灵 :我们来对古诗吧
小爱 : 好啊


运行之后会发现天猫精灵说出了“小爱同学”之后就没有了响应，这就是在使用条件变量的时候需要注意的点。仔细观察主函数中的线程启动顺序，tianmao先启动了，假设tianmao已经启动完成，并打印了“小爱同学”，执行notify之后，xiaoai才刚刚启动，成功执行完self.cond.acquire()之后，开始执行wait语句，但此时会陷入死循环！原因是 wait()只能被notify()唤醒，而notify()已经被另一个线程执行过了，注意：**只能是一个线程执行过了wait()，在被阻塞过程中，另一个线程执行了notify()才可以**。不然就像上面一下陷入死循环。因此，需要将上面的main方法改写：

```
if __name__ == "__main__":
    con = threading.Condition()
    xiaoai = XiaoAi(con)
    tianmao = TianMao(con)
    xiaoai.start()
    tianmao.start()

```
启动顺序很重要


进入wait()
1. 如果当前线程，没有获取到底层锁，那么抛出异常
2. 创建一个等待锁，并获取它，然后把它放到waiting池 (创建并获取[等待锁])
3. 释放底层锁，并保存锁对象的内部状态 (释放【底层锁】)

离开wait()
1. 获取[等待锁]（因此当前线程已经获取了一次该锁，因此当前线程会阻塞，直到其他线程释放该锁）
2. notify()释放[等待锁]后，获取【底层锁】，并重置锁对象的内部状态


notify()
1. 如果当前线程，没有获取到底层锁，那么抛出异常
2. 唤醒waiting池中前n个线程，并将他们从waiting池中移除(释放等待锁)
3. 当 waiting池中为空时，notify()返回 None

In [55]:
import threading,time

class XiaoAi(threading.Thread):
    def __init__(self, cond):
        super().__init__(name='小爱同学') # threading.Thread类中有 name 属性
        self.cond = cond
        
    def run(self):
        with self.cond: 
            print('1. {} : 获取条件变量的【底层锁】'.format(self.name))
            print('   {} : 等待'.format(self.name))
            print('2. {} : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】'.format(self.name))
            self.cond.wait() 
            print('6. {} : 离开 wait() - 线程在被唤醒或超时之后，并获取条件变量的【底层锁】'.format(self.name))
            print('   {} : 通知'.format(self.name))
            self.cond.notify()  
            print('7. {} : 释放[等待锁],唤醒waiting池中的线程'.format(self.name))
            print('   {} : 等待'.format(self.name))
            print("8. {} : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】".format(self.name))
            self.cond.wait()
            print("12.{} : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】".format(self.name))
            print('   {} : 通知'.format(self.name))
            self.cond.notify() 
            print('13.{} : 释放[等待锁],唤醒waiting池中的线程'.format(self.name))


class TianMao(threading.Thread):
    def __init__(self, cond):
        super().__init__(name='天猫精灵')
        self.cond = cond
        
    def run(self):
        with self.cond: 
            print('3. {} : 获取条件变量的【底层锁】'.format(self.name))
            print('   {} : 通知'.format(self.name))
            self.cond.notify() 
            print('4. {} : 释放[等待锁],唤醒waiting池中的线程'.format(self.name))
            print('   {} : 等待'.format(self.name))
            print('5. {} : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】'.format(self.name))
            self.cond.wait() 
            print('9. {} : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】, 并重置[等待锁]的内部状态'.format(self.name))
            print('   {} : 通知'.format(self.name))
            self.cond.notify() 
            print('10.{} : 释放[等待锁],唤醒waiting池中的线程'.format(self.name))
            print('   {} : 等待'.format(self.name))
            print('11.{} : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】'.format(self.name))
            self.cond.wait() 
            print('14.{} : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】'.format(self.name))


if __name__ == "__main__":
    con = threading.Condition()
    xiaoai = XiaoAi(con)
    tianmao = TianMao(con)
    xiaoai.start()
    tianmao.start()

1. 小爱同学 : 获取条件变量的【底层锁】
   小爱同学 : 等待
2. 小爱同学 : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】
3. 天猫精灵 : 获取条件变量的【底层锁】
   天猫精灵 : 通知
4. 天猫精灵 : 释放[等待锁],唤醒waiting池中的线程
   天猫精灵 : 等待
5. 天猫精灵 : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】
6. 小爱同学 : 离开 wait() - 线程在被唤醒或超时之后，并获取条件变量的【底层锁】
   小爱同学 : 通知
7. 小爱同学 : 释放[等待锁],唤醒waiting池中的线程
   小爱同学 : 等待
8. 小爱同学 : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】
9. 天猫精灵 : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】, 并重置[等待锁]的内部状态
   天猫精灵 : 通知
10.天猫精灵 : 释放[等待锁],唤醒waiting池中的线程
   天猫精灵 : 等待
11.天猫精灵 : 进入wait() - 线程在进入waiting池之后，创建并获取[等待锁]，并释放条件变量的【底层锁】
12.小爱同学 : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】
   小爱同学 : 通知
13.小爱同学 : 释放[等待锁],唤醒waiting池中的线程
14.天猫精灵 : 离开 wait() -  线程在被唤醒或超时之后，并获取条件变量的【底层锁】


### [Python Condition 源码分析](http://timd.cn/python/threading/condition/)

Condition（条件变量）可以看成是一个具有waiting池的RLock说明1（或Lock）。使用条件变量的流程通常是：

In [ ]:
from threading import Condition, Thread


class Storage(object):
    def __init__(self, max_size):
        self._max_size = max_size
        self._storage = []

    def is_full(self):
        return len(self._storage) == self._max_size

    def append(self, element):
        self._storage.append(element)

    def is_empty(self):
        return len(self._storage) == 0

    def pop(self):
        return self._storage.pop(0)


# 条件变量
condition = Condition()
max_count = 100
# 共享数据结构
storage = Storage(10)


def producer():
    global condition, max_count, storage
    current_count = 0

    while current_count < max_count:
        # 1，获取条件变量的底层锁
        condition.acquire()

        # 2，如果条件不满足，则进入waiting池，等待被唤醒
        if storage.is_full():
            # 需要特别说明的是：
            # 1，线程在进入waiting池之后，会释放条件变量的底层锁
            # 2，线程在被唤醒或超时之后，会获取条件变量的底层锁，
            #  + 也就是说，一旦离开wait()方法，那么该线程就持有了底层锁
            print("produce thread enter into waiting pool")
            condition.wait()
            print("produce thread is waken up")
            # 2.1，被唤醒之后，继续进行判断
        # 3，如果条件满足，则更新共享数据
        else:
            print("produce: %d" % current_count)
            storage.append(current_count)
            current_count = current_count + 1
            # 3.1，唤醒waiting池中的线程
            condition.notify()
        # 释放条件变量的底层锁
        condition.release()


def consumer():
    global condition, max_count, storage
    current_count = 0

    while current_count < max_count:
        condition.acquire()
        if storage.is_empty():
            print("consume thread enter into waiting pool")
            condition.wait()
            print("consume thread is waken up")
        else:
            element = storage.pop()
            print("consume: %d" % element)
            current_count = current_count + 1
            condition.notify()
        condition.release()


consumer_thread = Thread(target=consumer)
consumer_thread.start()

producer_thread = Thread(target=producer)
producer_thread.start()

consumer_thread.join()
producer_thread.join()

主要源码,与Condition相关的代码:

In [ ]:
def Condition(*args, **kwargs):
    return _Condition(*args, **kwargs)

class _Condition(_Verbose):
    # 条件变量允许一个或多个线程进入到等待状态，直到它们被其他线程唤醒
    """Condition variables allow one or more threads to wait until they are
       notified by another thread.
    """

    def __init__(self, lock=None, verbose=None):
        ...
        if lock is None:
            lock = RLock()
        # Condition的底层锁，默认是RLock
        self.__lock = lock
        # 将Condition的acquire()和release()方法设置为底层锁的acquire()和release()方法
        self.acquire = lock.acquire
        self.release = lock.release

        try:
            self._release_save = lock._release_save
        except AttributeError:
            pass
        try:
            self._acquire_restore = lock._acquire_restore
        except AttributeError:
            pass

        try:
            self._is_owned = lock._is_owned
        except AttributeError:
            pass
        # waiting池
        self.__waiters = []

    def _is_owned(self):
        # 如果当前线程持有锁，则返回True，否则返回False
        # 说明2
        # 其过程是：使用非阻塞的方式获取锁，如果获取成功，则表明当前线程，
        # + 没有持有锁，那么释放获取到的锁，并返回False；
        # + + 否则，认为当前线程持有锁，返回True。

        # 只有当底层锁没有_is_owned()方法时，才会用这种方式判断当前线程是否拥有底层锁
        # 因为RLock具有_is_owned()方法，所以，它的对象不会使用这里的_is_owned()方法
        if self.__lock.acquire(0):
            self.__lock.release()
            return False
        else:
            return True

    # Condition也支持上下文管理。
    # + 当进入到Condtion对象时，会获取底层锁
    # + 当离开到Condtion对象时，会释放底层锁
    def __enter__(self):
        return self.__lock.__enter__()

    def __exit__(self, *args):
        return self.__lock.__exit__(*args)


    # 对于RLock而言，调用release()方法，并不一定会真正的释放锁。
    # + 因此，它提供了_release_save()方法，该方法会真正的释放锁，
    # + + 并且将RLock内部维护的状态，返回给调用方。
    # 之后，线程再次获取到底层锁之后，再将状态重置回去
    # RLock内部会维护两个状态：owner-拥有锁的线程的id，count-该线程获取了多少次锁

    # 只有底层锁没有_release_save()和_acquire_restore()方法时，才用下面的实现
    def _release_save(self):
        self.__lock.release()           # No state to save

    def _acquire_restore(self, x):
        self.__lock.acquire()           # Ignore saved state


    def wait(self, timeout=None):
        # 1，如果当前线程，没有获取到底层锁，那么抛出异常
        if not self._is_owned():
            raise RuntimeError("cannot wait on un-acquired lock")

        # 2，创建一个锁对象，并获取它，然后把它放到waiting池
        waiter = _allocate_lock()
        waiter.acquire()
        self.__waiters.append(waiter)

        # 3，释放底层锁，并保存锁对象的内部状态
        saved_state = self._release_save()
        try:    # restore state no matter what (e.g., KeyboardInterrupt)
            if timeout is None:
                # 3.1，如果timeout是None，那么再次以阻塞的方式获取锁对象
                #  + 因此当前线程已经获取了一次该锁，因此当前线程会阻塞，直到其他线程释放该锁
                waiter.acquire() 
                if __debug__:
                    self._note("%s.wait(): got it", self)
            else:
                # 3.2，如果timeout不是None，那么重复下面的流程：
                #  + 1，以非阻塞方式获取锁
                #  + + 1.1，如果获取成功，说明其他线程释放了该锁，那么退出循环
                #  + + 1.2，如果获取失败，那么看是否达到了超时时间，如果达到了，那么退出循环；否则，继续
                #  + 2，sleep一小段时间，然后回到步骤1
                #  + + 每次循环sleep的时长，是从0.0005秒开始，指数增长，最多增长到0.05秒
                #  + + + 第一次是：0.0005，第二次是：0.001，...
                endtime = _time() + timeout
                delay = 0.0005 # 500 us -> initial delay of 1 ms
                while True:
                    gotit = waiter.acquire(0)
                    if gotit:
                        break
                    remaining = endtime - _time()
                    if remaining <= 0:
                        break
                    delay = min(delay * 2, remaining, .05)
                    _sleep(delay)
                if not gotit:
                    ...
                    try:
                        # 3.3，如果因为超时，而不是被唤醒，退出的wait()，那么将锁从waiting池中移除
                        # + 因为当线程被唤醒的时候，调用notify的线程会将锁从waiting池中移除
                        self.__waiters.remove(waiter)
                    except ValueError:
                        pass
                else:
                    ...
        finally:
            # 4，获取底层锁，并重置锁对象的内部状态
            self._acquire_restore(saved_state)

    def notify(self, n=1):
        """Wake up one or more threads waiting on this condition, if any.

        If the calling thread has not acquired the lock when this method is
        called, a RuntimeError is raised.

        This method wakes up at most n of the threads waiting for the condition
        variable; it is a no-op if no threads are waiting.

        """        
        # 1，如果当前线程，没有获取到底层锁，那么抛出异常
        if not self._is_owned():
            raise RuntimeError("cannot notify on un-acquired lock")

        __waiters = self.__waiters
        waiters_to_notify = _deque(_islice(all_waiters, n))
        if not waiters_to_notify: # 当 waiting池中为空，返回 None
            return

        # 2，唤醒waiting池中前n个线程，并将他们从waiting池中移除
        for waiter in waiters_to_notify:
            waiter.release()
            try:
                all_waiters.remove(waiter)
            except ValueError:
                pass

    def notify_all(self):
        """Wake up all threads waiting on this condition.

        If the calling thread has not acquired the lock when this method
        is called, a RuntimeError is raised.

        """
        self.notify(len(self._waiters))

    notifyAll = notify_all

生产者与消费者

实现场景：当a同学王火锅里面添加鱼丸加满后（最多5个，加满后通知b去吃掉），通知b同学去吃掉鱼丸（吃到0的时候通知a同学继续添加）

#### 两个线程之间用 condition 实现线程同步

In [2]:
import threading
import time

con = threading.Condition()

num = 0

# 生产者
class Producer(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)
        self.name = 'Producer'

    def run(self):

        global num
        con.acquire()
        print('1.{}: 获取底层锁'.format(self.name))
        for i in range(5):
            print ("开始添加！！！")
            num += 1
            print ("火锅里面鱼丸个数：%s" % str(num))
            time.sleep(1)
            if num >= 5:
                print( "火锅里面里面鱼丸数量已经到达5个，无法添加了！")

                con.notify()  # 当 waiting池中为空时，notify()返回 None
                print('2.{}: 进入wait() 创建并获取[等待锁]  释放底层锁'.format(self.name))
                con.wait()
                print('6.{}: 离开wait() 获取[等待锁] 获取【底层锁】 '.format(self.name))
        # 释放锁
        con.release()
        print('7.{}: 释放底层锁'.format(self.name))

# 消费者
class Consumers(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        self.name = 'Consumers'

    def run(self):
        con.acquire()
        print('3.{}: 获取底层锁'.format(self.name))
        global num
        for i in range(5):
            print( "开始吃啦！！！")
            num -= 1
            print ("火锅里面剩余鱼丸数量：%s" %str(num))
            time.sleep(2)
            if num <= 0:
                print( "锅底没货了，赶紧加鱼丸吧！")
                print('4.{}: 唤醒waiting池中线程 释放等待锁'.format(self.name))
                con.notify()
                print('5.{}: 进入wait() 创建并获取[等待锁]  释放底层锁'.format(self.name))
                con.wait()
        #con.release()
        #print('8.{}: 释放底层锁'.format(self.name))

p = Producer()
c = Consumers()
p.start()
c.start()

1.Producer: 获取底层锁
开始添加！！！
火锅里面鱼丸个数：1
开始添加！！！
火锅里面鱼丸个数：2
开始添加！！！
火锅里面鱼丸个数：3
开始添加！！！
火锅里面鱼丸个数：4
开始添加！！！
火锅里面鱼丸个数：5
火锅里面里面鱼丸数量已经到达5个，无法添加了！
2.Producer: 进入wait() 创建并获取[等待锁]  释放底层锁
3.Consumers: 获取底层锁
开始吃啦！！！
火锅里面剩余鱼丸数量：4
开始吃啦！！！
火锅里面剩余鱼丸数量：3
开始吃啦！！！
火锅里面剩余鱼丸数量：2
开始吃啦！！！
火锅里面剩余鱼丸数量：1
开始吃啦！！！
火锅里面剩余鱼丸数量：0
锅底没货了，赶紧加鱼丸吧！
4.Consumers: 唤醒waiting池中线程 释放等待锁
5.Consumers: 进入wait() 创建并获取[等待锁]  释放底层锁
6.Producer: 离开wait() 获取[等待锁] 获取【底层锁】 
7.Producer: 释放底层锁


#### 多个线程之间用 condition 实现线程同步

下面创建了5个线程，当一个线程中的 condition 底层锁被释放的时候，所有线程开始抢 condition 底层锁，谁抢到谁运行。
结果显示，没有time.sleep(1)时，刚放出的锁就被自己抢了。如果注释掉time.sleep(1)，结果会比较有顺序；如果使用 time.sleep(random.random())，结果就会无序了。

In [1]:
import time,random,threading

queue = [] 
con = threading.Condition()
 
class Producer(threading.Thread):
    def run(self):
        while True:
            if con.acquire():
                if len(queue) > 5:
                    con.wait()
                else:
                    elem = random.randrange(100)
                    queue.append(elem)
                    print("{}: Producer a elem {}, Now size is {}".format(self.name,elem, len(queue)))
                    con.notify()
                con.release()
                #time.sleep(1)
                
class Consumer(threading.Thread):
    def run(self):
        while True:
            if con.acquire():
                if len(queue) <= 0:
                    con.wait()
                else:
                    elem = queue.pop()
                    print("{}: Consumer a elem {}. Now size is {}".format(self.name,elem, len(queue)))
                    con.notify()
                con.release()
                #time.sleep(1)

def main():
    for i in range(3):
        Producer().start()
    for i in range(2):
        Consumer().start()

main()

Thread-6: Producer a elem 30, Now size is 1
Thread-6: Producer a elem 22, Now size is 2
Thread-6: Producer a elem 46, Now size is 3
Thread-6: Producer a elem 16, Now size is 4
Thread-6: Producer a elem 77, Now size is 5
Thread-6: Producer a elem 40, Now size is 6
Thread-9: Consumer a elem 40. Now size is 5
Thread-9: Consumer a elem 77. Now size is 4
Thread-9: Consumer a elem 16. Now size is 3
Thread-9: Consumer a elem 46. Now size is 2
Thread-9: Consumer a elem 22. Now size is 1
Thread-9: Consumer a elem 30. Now size is 0
Thread-7: Producer a elem 7, Now size is 1
Thread-7: Producer a elem 8, Now size is 2
Thread-7: Producer a elem 22, Now size is 3
Thread-7: Producer a elem 69, Now size is 4
Thread-7: Producer a elem 85, Now size is 5
Thread-7: Producer a elem 75, Now size is 6
Thread-9: Consumer a elem 75. Now size is 5
Thread-9: Consumer a elem 85. Now size is 4
Thread-9: Consumer a elem 69. Now size is 3
Thread-9: Consumer a elem 22. Now size is 2
Thread-9: Consumer a elem 8. Now s

Thread-6: Producer a elem 18, Now size is 1
Thread-6: Producer a elem 18, Now size is 2
Thread-6: Producer a elem 33, Now size is 3
Thread-6: Producer a elem 63, Now size is 4
Thread-6: Producer a elem 55, Now size is 5
Thread-6: Producer a elem 61, Now size is 6
Thread-10: Consumer a elem 61. Now size is 5
Thread-10: Consumer a elem 55. Now size is 4
Thread-10: Consumer a elem 63. Now size is 3
Thread-10: Consumer a elem 33. Now size is 2
Thread-10: Consumer a elem 18. Now size is 1
Thread-10: Consumer a elem 18. Now size is 0
Thread-8: Producer a elem 9, Now size is 1
Thread-8: Producer a elem 68, Now size is 2
Thread-8: Producer a elem 87, Now size is 3
Thread-8: Producer a elem 43, Now size is 4
Thread-8: Producer a elem 8, Now size is 5
Thread-8: Producer a elem 5, Now size is 6
Thread-9: Consumer a elem 5. Now size is 5
Thread-9: Consumer a elem 8. Now size is 4
Thread-9: Consumer a elem 43. Now size is 3
Thread-9: Consumer a elem 87. Now size is 2
Thread-9: Consumer a elem 68. N

Thread-9: Consumer a elem 77. Now size is 4
Thread-9: Consumer a elem 64. Now size is 3
Thread-9: Consumer a elem 83. Now size is 2
Thread-9: Consumer a elem 1. Now size is 1
Thread-9: Consumer a elem 15. Now size is 0
Thread-8: Producer a elem 15, Now size is 1
Thread-8: Producer a elem 1, Now size is 2
Thread-8: Producer a elem 35, Now size is 3
Thread-8: Producer a elem 9, Now size is 4
Thread-8: Producer a elem 53, Now size is 5
Thread-8: Producer a elem 45, Now size is 6
Thread-9: Consumer a elem 45. Now size is 5
Thread-9: Consumer a elem 53. Now size is 4
Thread-9: Consumer a elem 9. Now size is 3
Thread-9: Consumer a elem 35. Now size is 2
Thread-9: Consumer a elem 1. Now size is 1
Thread-9: Consumer a elem 15. Now size is 0
Thread-7: Producer a elem 91, Now size is 1
Thread-7: Producer a elem 30, Now size is 2
Thread-7: Producer a elem 69, Now size is 3
Thread-7: Producer a elem 83, Now size is 4
Thread-7: Producer a elem 55, Now size is 5
Thread-7: Producer a elem 72, Now siz

Thread-10: Consumer a elem 9. Now size is 2
Thread-10: Consumer a elem 15. Now size is 1
Thread-10: Consumer a elem 58. Now size is 0
Thread-6: Producer a elem 83, Now size is 1
Thread-6: Producer a elem 85, Now size is 2
Thread-6: Producer a elem 67, Now size is 3
Thread-6: Producer a elem 62, Now size is 4
Thread-6: Producer a elem 69, Now size is 5
Thread-6: Producer a elem 69, Now size is 6
Thread-9: Consumer a elem 69. Now size is 5
Thread-9: Consumer a elem 69. Now size is 4
Thread-9: Consumer a elem 62. Now size is 3
Thread-9: Consumer a elem 67. Now size is 2
Thread-9: Consumer a elem 85. Now size is 1
Thread-9: Consumer a elem 83. Now size is 0
Thread-8: Producer a elem 20, Now size is 1
Thread-8: Producer a elem 37, Now size is 2
Thread-8: Producer a elem 36, Now size is 3
Thread-8: Producer a elem 47, Now size is 4
Thread-8: Producer a elem 33, Now size is 5
Thread-8: Producer a elem 83, Now size is 6
Thread-10: Consumer a elem 83. Now size is 5
Thread-10: Consumer a elem 33

Thread-6: Producer a elem 13, Now size is 1
Thread-6: Producer a elem 11, Now size is 2
Thread-6: Producer a elem 56, Now size is 3
Thread-6: Producer a elem 93, Now size is 4
Thread-6: Producer a elem 20, Now size is 5
Thread-6: Producer a elem 7, Now size is 6
Thread-10: Consumer a elem 7. Now size is 5
Thread-10: Consumer a elem 20. Now size is 4
Thread-10: Consumer a elem 93. Now size is 3
Thread-10: Consumer a elem 56. Now size is 2
Thread-10: Consumer a elem 11. Now size is 1
Thread-10: Consumer a elem 13. Now size is 0
Thread-6: Producer a elem 74, Now size is 1
Thread-6: Producer a elem 45, Now size is 2
Thread-6: Producer a elem 92, Now size is 3
Thread-6: Producer a elem 98, Now size is 4
Thread-10: Consumer a elem 98. Now size is 3
Thread-10: Consumer a elem 92. Now size is 2
Thread-10: Consumer a elem 45. Now size is 1
Thread-10: Consumer a elem 74. Now size is 0
Thread-8: Producer a elem 83, Now size is 1
Thread-8: Producer a elem 77, Now size is 2
Thread-8: Producer a ele

...

Thread-7: Producer a elem 29, Now size is 1
Thread-7: Producer a elem 48, Now size is 2
Thread-7: Producer a elem 13, Now size is 3
Thread-7: Producer a elem 77, Now size is 4
Thread-7: Producer a elem 15, Now size is 5
Thread-7: Producer a elem 97, Now size is 6
Thread-10: Consumer a elem 97. Now size is 5
Thread-10: Consumer a elem 15. Now size is 4
Thread-10: Consumer a elem 77. Now size is 3
Thread-10: Consumer a elem 13. Now size is 2
Thread-10: Consumer a elem 48. Now size is 1
Thread-10: Consumer a elem 29. Now size is 0
Thread-8: Producer a elem 76, Now size is 1
Thread-8: Producer a elem 70, Now size is 2
Thread-8: Producer a elem 94, Now size is 3
Thread-8: Producer a elem 75, Now size is 4
Thread-8: Producer a elem 50, Now size is 5
Thread-8: Producer a elem 93, Now size is 6
Thread-10: Consumer a elem 93. Now size is 5
Thread-10: Consumer a elem 50. Now size is 4
Thread-10: Consumer a elem 75. Now size is 3
Thread-10: Consumer a elem 94. Now size is 2
Thread-10: Consumer a 

Thread-9: Consumer a elem 61. Now size is 4
Thread-9: Consumer a elem 23. Now size is 3
Thread-9: Consumer a elem 74. Now size is 2
Thread-9: Consumer a elem 99. Now size is 1
Thread-9: Consumer a elem 82. Now size is 0
Thread-6: Producer a elem 58, Now size is 1
Thread-6: Producer a elem 49, Now size is 2
Thread-6: Producer a elem 4, Now size is 3
Thread-6: Producer a elem 46, Now size is 4
Thread-6: Producer a elem 92, Now size is 5
Thread-6: Producer a elem 48, Now size is 6
Thread-10: Consumer a elem 48. Now size is 5
Thread-10: Consumer a elem 92. Now size is 4
Thread-10: Consumer a elem 46. Now size is 3
Thread-10: Consumer a elem 4. Now size is 2
Thread-10: Consumer a elem 49. Now size is 1
Thread-10: Consumer a elem 58. Now size is 0
Thread-7: Producer a elem 60, Now size is 1
Thread-7: Producer a elem 70, Now size is 2
Thread-7: Producer a elem 90, Now size is 3
Thread-7: Producer a elem 62, Now size is 4
Thread-7: Producer a elem 51, Now size is 5
Thread-7: Producer a elem 66

Thread-7: Producer a elem 53, Now size is 1
Thread-7: Producer a elem 18, Now size is 2
Thread-7: Producer a elem 19, Now size is 3
Thread-7: Producer a elem 56, Now size is 4
Thread-7: Producer a elem 68, Now size is 5
Thread-7: Producer a elem 20, Now size is 6
Thread-9: Consumer a elem 20. Now size is 5
Thread-9: Consumer a elem 68. Now size is 4
Thread-9: Consumer a elem 56. Now size is 3
Thread-9: Consumer a elem 19. Now size is 2
Thread-9: Consumer a elem 18. Now size is 1
Thread-9: Consumer a elem 53. Now size is 0
Thread-6: Producer a elem 54, Now size is 1
Thread-6: Producer a elem 26, Now size is 2
Thread-6: Producer a elem 49, Now size is 3
Thread-6: Producer a elem 23, Now size is 4
Thread-6: Producer a elem 15, Now size is 5
Thread-6: Producer a elem 36, Now size is 6
Thread-10: Consumer a elem 36. Now size is 5
Thread-10: Consumer a elem 15. Now size is 4
Thread-10: Consumer a elem 23. Now size is 3
Thread-10: Consumer a elem 49. Now size is 2
Thread-10: Consumer a elem 2

Thread-10: Consumer a elem 13. Now size is 5
Thread-10: Consumer a elem 19. Now size is 4
Thread-10: Consumer a elem 27. Now size is 3
Thread-10: Consumer a elem 95. Now size is 2
Thread-10: Consumer a elem 13. Now size is 1
Thread-10: Consumer a elem 33. Now size is 0
Thread-7: Producer a elem 87, Now size is 1
Thread-7: Producer a elem 40, Now size is 2
Thread-7: Producer a elem 54, Now size is 3
Thread-7: Producer a elem 39, Now size is 4
Thread-7: Producer a elem 72, Now size is 5
Thread-7: Producer a elem 11, Now size is 6
Thread-10: Consumer a elem 11. Now size is 5
Thread-10: Consumer a elem 72. Now size is 4
Thread-10: Consumer a elem 39. Now size is 3
Thread-10: Consumer a elem 54. Now size is 2
Thread-10: Consumer a elem 40. Now size is 1
Thread-10: Consumer a elem 87. Now size is 0
Thread-7: Producer a elem 85, Now size is 1
Thread-7: Producer a elem 0, Now size is 2
Thread-7: Producer a elem 44, Now size is 3
Thread-7: Producer a elem 32, Now size is 4
Thread-7: Producer a 

Thread-10: Consumer a elem 91. Now size is 5
Thread-10: Consumer a elem 32. Now size is 4
Thread-10: Consumer a elem 37. Now size is 3
Thread-10: Consumer a elem 94. Now size is 2
Thread-10: Consumer a elem 74. Now size is 1
Thread-10: Consumer a elem 54. Now size is 0
Thread-8: Producer a elem 19, Now size is 1
Thread-8: Producer a elem 11, Now size is 2
Thread-8: Producer a elem 69, Now size is 3
Thread-8: Producer a elem 33, Now size is 4
Thread-8: Producer a elem 25, Now size is 5
Thread-8: Producer a elem 66, Now size is 6
Thread-9: Consumer a elem 66. Now size is 5
Thread-9: Consumer a elem 25. Now size is 4
Thread-9: Consumer a elem 33. Now size is 3
Thread-9: Consumer a elem 69. Now size is 2
Thread-9: Consumer a elem 11. Now size is 1
Thread-9: Consumer a elem 19. Now size is 0
Thread-7: Producer a elem 20, Now size is 1
Thread-7: Producer a elem 77, Now size is 2
Thread-7: Producer a elem 16, Now size is 3
Thread-7: Producer a elem 13, Now size is 4
Thread-7: Producer a elem 

上述就是一个简单的生产者消费模型，先看生产者，生产者条件变量锁之后就检查条件，如果不符合条件则wait，wait的时候会释放锁。如果条件符合，则往队列添加元素，然后会notify其他线程。注意生产者调用了condition的notify()方法后，消费者被唤醒，但唤醒不意味着它可以开始运行，notify()并不释放lock，调用notify()后，lock依然被生产者所持有。生产者通过con.release()显式释放lock。消费者再次开始运行，获得条件锁然后判断条件执行。

### 队列

生产消费者模型主要是对队列进程操作，贴心的Python为我们实现了一个队列结构，队列内部实现了锁的相关设置。可以用队列重写生产消费者模型。

In [4]:
import time,random,threading,queue

q = queue.Queue(4)

class Producer(threading.Thread):
    def run(self):
        while True:
            elem = random.randrange(100)
            q.put(elem)
            print("{}: Producer a elem {}, Now size is {}".format(self.name,elem, q.qsize()))

                
class Consumer(threading.Thread):
    def run(self):
        while True:
            elem = q.get()
            q.task_done()
            print("{}: Consumer a elem {}. Now size is {}".format(self.name,elem, q.qsize()))


def main():
    for i in range(3):
        Producer().start()
    for i in range(2):
        Consumer().start()

main()

Thread-21: Producer a elem 66, Now size is 1
Thread-21: Producer a elem 47, Now size is 2
Thread-21: Producer a elem 51, Now size is 3
Thread-21: Producer a elem 59, Now size is 4
Thread-24: Consumer a elem 66. Now size is 3Thread-21: Producer a elem 28, Now size is 4

Thread-25: Consumer a elem 47. Now size is 3Thread-24: Consumer a elem 51. Now size is 2
Thread-24: Consumer a elem 59. Now size is 1
Thread-24: Consumer a elem 28. Now size is 0
Thread-21: Producer a elem 67, Now size is 1Thread-22: Producer a elem 74, Now size is 2

Thread-23: Producer a elem 15, Now size is 3Thread-21: Producer a elem 56, Now size is 4Thread-25: Consumer a elem 67. Now size is 3
Thread-22: Producer a elem 60, Now size is 4

Thread-25: Consumer a elem 74. Now size is 3
Thread-25: Consumer a elem 15. Now size is 2
Thread-25: Consumer a elem 56. Now size is 1
Thread-25: Consumer a elem 60. Now size is 0
Thread-22: Producer a elem 44, Now size is 1
Thread-22: Producer a elem 51, Now size is 2
Thread-22: P

Thread-24: Consumer a elem 48. Now size is 3

Thread-22: Producer a elem 45, Now size is 4

Thread-24: Consumer a elem 91. Now size is 3
Thread-24: Consumer a elem 58. Now size is 2
Thread-24: Consumer a elem 96. Now size is 1
Thread-24: Consumer a elem 45. Now size is 0
Thread-23: Producer a elem 16, Now size is 1Thread-21: Producer a elem 10, Now size is 2Thread-22: Producer a elem 33, Now size is 3
Thread-22: Producer a elem 71, Now size is 4

Thread-24: Consumer a elem 16. Now size is 3
Thread-24: Consumer a elem 10. Now size is 2
Thread-24: Consumer a elem 33. Now size is 1
Thread-24: Consumer a elem 71. Now size is 0
Thread-21: Producer a elem 82, Now size is 1
Thread-21: Producer a elem 25, Now size is 2
Thread-21: Producer a elem 24, Now size is 3
Thread-21: Producer a elem 29, Now size is 4
Thread-24: Consumer a elem 82. Now size is 3
Thread-24: Consumer a elem 25. Now size is 2
Thread-24: Consumer a elem 24. Now size is 1
Thread-24: Consumer a elem 29. Now size is 0

Thread-2

Thread-25: Consumer a elem 55. Now size is 3
Thread-25: Consumer a elem 22. Now size is 2
Thread-25: Consumer a elem 2. Now size is 1
Thread-25: Consumer a elem 60. Now size is 0
Thread-21: Producer a elem 35, Now size is 1Thread-24: Consumer a elem 35. Now size is 0

Thread-21: Producer a elem 78, Now size is 1
Thread-21: Producer a elem 9, Now size is 2
Thread-21: Producer a elem 41, Now size is 3
Thread-21: Producer a elem 64, Now size is 4
Thread-25: Consumer a elem 78. Now size is 3Thread-24: Consumer a elem 9. Now size is 2
Thread-25: Consumer a elem 41. Now size is 1
Thread-25: Consumer a elem 64. Now size is 0
Thread-22: Producer a elem 86, Now size is 1
Thread-24: Consumer a elem 86. Now size is 0

Thread-22: Producer a elem 62, Now size is 1
Thread-22: Producer a elem 48, Now size is 2
Thread-22: Producer a elem 22, Now size is 3
Thread-22: Producer a elem 52, Now size is 4
Thread-24: Consumer a elem 62. Now size is 3
Thread-24: Consumer a elem 48. Now size is 2
Thread-24: Co

Thread-25: Consumer a elem 78. Now size is 3
Thread-21: Producer a elem 24, Now size is 4

Thread-25: Consumer a elem 67. Now size is 3
Thread-25: Consumer a elem 44. Now size is 2
Thread-25: Consumer a elem 44. Now size is 1
Thread-25: Consumer a elem 24. Now size is 0
Thread-21: Producer a elem 29, Now size is 1
Thread-21: Producer a elem 23, Now size is 2
Thread-21: Producer a elem 88, Now size is 3
Thread-21: Producer a elem 78, Now size is 4
Thread-25: Consumer a elem 29. Now size is 3
Thread-25: Consumer a elem 23. Now size is 2
Thread-25: Consumer a elem 88. Now size is 1
Thread-25: Consumer a elem 78. Now size is 0
Thread-21: Producer a elem 13, Now size is 1
Thread-21: Producer a elem 12, Now size is 2
Thread-21: Producer a elem 35, Now size is 3
Thread-21: Producer a elem 62, Now size is 4
Thread-25: Consumer a elem 13. Now size is 3
Thread-25: Consumer a elem 12. Now size is 2
Thread-23: Producer a elem 23, Now size is 3Thread-22: Producer a elem 20, Now size is 4Thread-24: 

Thread-21: Producer a elem 63, Now size is 1Thread-22: Producer a elem 22, Now size is 2Thread-23: Producer a elem 39, Now size is 3Thread-24: Consumer a elem 63. Now size is 2Thread-25: Consumer a elem 22. Now size is 1

Thread-22: Producer a elem 21, Now size is 2
Thread-22: Producer a elem 91, Now size is 3
Thread-22: Producer a elem 32, Now size is 4


Thread-25: Consumer a elem 39. Now size is 3
Thread-25: Consumer a elem 21. Now size is 2
Thread-25: Consumer a elem 91. Now size is 1
Thread-25: Consumer a elem 32. Now size is 0
Thread-23: Producer a elem 73, Now size is 1
Thread-23: Producer a elem 15, Now size is 2
Thread-23: Producer a elem 6, Now size is 3
Thread-23: Producer a elem 49, Now size is 4

Thread-24: Consumer a elem 73. Now size is 3
Thread-24: Consumer a elem 15. Now size is 2
Thread-24: Consumer a elem 6. Now size is 1
Thread-24: Consumer a elem 49. Now size is 0
Thread-23: Producer a elem 26, Now size is 1
Thread-23: Producer a elem 97, Now size is 2
Thread-23: P

...

Thread-23: Producer a elem 13, Now size is 1Thread-22: Producer a elem 57, Now size is 2Thread-24: Consumer a elem 13. Now size is 1
Thread-23: Producer a elem 51, Now size is 2
Thread-23: Producer a elem 79, Now size is 3
Thread-23: Producer a elem 1, Now size is 4

Thread-24: Consumer a elem 57. Now size is 3
Thread-24: Consumer a elem 51. Now size is 2
Thread-24: Consumer a elem 79. Now size is 1
Thread-24: Consumer a elem 1. Now size is 0

Thread-21: Producer a elem 95, Now size is 1
Thread-21: Producer a elem 8, Now size is 2
Thread-21: Producer a elem 55, Now size is 3
Thread-21: Producer a elem 32, Now size is 4
Thread-24: Consumer a elem 95. Now size is 3
Thread-24: Consumer a elem 8. Now size is 2
Thread-24: Consumer a elem 55. Now size is 1
Thread-24: Consumer a elem 32. Now size is 0
Thread-21: Producer a elem 47, Now size is 1
Thread-21: Producer a elem 22, Now size is 2
Thread-21: Producer a elem 48, Now size is 3
Thread-21: Producer a elem 3, Now size is 4
Thread-24: Cons


Thread-23: Producer a elem 56, Now size is 1
Thread-23: Producer a elem 37, Now size is 2

Thread-24: Consumer a elem 56. Now size is 1Thread-23: Producer a elem 31, Now size is 2
Thread-23: Producer a elem 79, Now size is 3
Thread-25: Consumer a elem 37. Now size is 2
Thread-25: Consumer a elem 31. Now size is 1
Thread-25: Consumer a elem 79. Now size is 0
Thread-21: Producer a elem 56, Now size is 1
Thread-24: Consumer a elem 56. Now size is 0

Thread-21: Producer a elem 93, Now size is 1
Thread-21: Producer a elem 91, Now size is 2
Thread-21: Producer a elem 57, Now size is 3
Thread-21: Producer a elem 19, Now size is 4
Thread-25: Consumer a elem 93. Now size is 3
Thread-24: Consumer a elem 91. Now size is 2Thread-21: Producer a elem 66, Now size is 3
Thread-21: Producer a elem 87, Now size is 4
Thread-25: Consumer a elem 57. Now size is 3
Thread-25: Consumer a elem 19. Now size is 2
Thread-25: Consumer a elem 66. Now size is 1
Thread-25: Consumer a elem 87. Now size is 0
Thread-23

Thread-24: Consumer a elem 63. Now size is 3
Thread-24: Consumer a elem 74. Now size is 2
Thread-24: Consumer a elem 13. Now size is 1
Thread-24: Consumer a elem 14. Now size is 0
Thread-22: Producer a elem 28, Now size is 1Thread-25: Consumer a elem 28. Now size is 0
Thread-22: Producer a elem 87, Now size is 1
Thread-22: Producer a elem 54, Now size is 2
Thread-22: Producer a elem 69, Now size is 3
Thread-22: Producer a elem 48, Now size is 4
Thread-24: Consumer a elem 87. Now size is 3
Thread-23: Producer a elem 68, Now size is 4Thread-24: Consumer a elem 54. Now size is 3
Thread-24: Consumer a elem 69. Now size is 2
Thread-24: Consumer a elem 48. Now size is 1
Thread-24: Consumer a elem 68. Now size is 0

Thread-23: Producer a elem 3, Now size is 1
Thread-22: Producer a elem 18, Now size is 2Thread-21: Producer a elem 98, Now size is 3
Thread-24: Consumer a elem 3. Now size is 2
Thread-24: Consumer a elem 18. Now size is 1
Thread-24: Consumer a elem 98. Now size is 0
Thread-23: Pro

Thread-21: Producer a elem 30, Now size is 1Thread-23: Producer a elem 19, Now size is 2Thread-22: Producer a elem 94, Now size is 3Thread-24: Consumer a elem 30. Now size is 2

Thread-22: Producer a elem 47, Now size is 3
Thread-22: Producer a elem 61, Now size is 4

Thread-24: Consumer a elem 19. Now size is 3

Thread-21: Producer a elem 99, Now size is 4
Thread-24: Consumer a elem 94. Now size is 3
Thread-24: Consumer a elem 47. Now size is 2
Thread-24: Consumer a elem 61. Now size is 1
Thread-24: Consumer a elem 99. Now size is 0
Thread-21: Producer a elem 45, Now size is 1
Thread-21: Producer a elem 54, Now size is 2
Thread-21: Producer a elem 24, Now size is 3
Thread-21: Producer a elem 3, Now size is 4
Thread-24: Consumer a elem 45. Now size is 3
Thread-24: Consumer a elem 54. Now size is 2
Thread-24: Consumer a elem 24. Now size is 1
Thread-24: Consumer a elem 3. Now size is 0
Thread-22: Producer a elem 93, Now size is 1
Thread-22: Producer a elem 97, Now size is 2
Thread-22: P

Thread-21: Producer a elem 1, Now size is 1
Thread-23: Producer a elem 1, Now size is 2Thread-24: Consumer a elem 1. Now size is 1
Thread-21: Producer a elem 77, Now size is 2
Thread-21: Producer a elem 85, Now size is 3
Thread-21: Producer a elem 13, Now size is 4


Thread-24: Consumer a elem 1. Now size is 3
Thread-24: Consumer a elem 77. Now size is 2
Thread-24: Consumer a elem 85. Now size is 1
Thread-24: Consumer a elem 13. Now size is 0
Thread-21: Producer a elem 3, Now size is 1
Thread-21: Producer a elem 47, Now size is 2
Thread-21: Producer a elem 50, Now size is 3
Thread-21: Producer a elem 26, Now size is 4

Thread-25: Consumer a elem 3. Now size is 3
Thread-25: Consumer a elem 47. Now size is 2
Thread-25: Consumer a elem 50. Now size is 1
Thread-25: Consumer a elem 26. Now size is 0
Thread-21: Producer a elem 72, Now size is 1
Thread-21: Producer a elem 46, Now size is 2
Thread-21: Producer a elem 92, Now size is 3
Thread-21: Producer a elem 11, Now size is 4
Thread-24: Con

queue内部实现了相关的锁，如果queue的为空，则get元素的时候会被阻塞，知道队列里面被其他线程写入数据。同理，当写入数据的时候，如果元素个数大于队列的长度，也会被阻塞。也就是在 put 或 get的时候都会获得Lock。

### [python队列模块Queue](http://www.361way.com/python-queue/4612.html)


####  初识Queue模块
Queue模块实现了多生产者、多消费者队列。它特别适用于信息必须在多个线程间安全地交换的多线程程序中。这个模块中的Queue类实现了所有必须的锁语义。它依赖于Python中线程支持的可用性；参见threading模块。

模块实现了三类队列：FIFO（First In First Out，先进先出，默认为该队列）、LIFO（Last In First Out，后进先出）、基于优先级的队列。以下为其常用方法：

```
先进先出  q = Queue.Queue(maxsize)
后进先出  a = Queue.LifoQueue(maxsize)
优先级  Queue.PriorityQueue(maxsize)
Queue.qsize() 返回队列的大小
Queue.empty() 如果队列为空，返回True,反之False
Queue.full() 如果队列满了，返回True,反之False
Queue.full 与 maxsize 大小对应
Queue.put(item) 写入队列，timeout等待时间   非阻塞
Queue.get([block[, timeout]]) 获取队列，timeout等待时间
Queue.get_nowait() 相当Queue.get(False)
Queue.put_nowait(item) 相当Queue.put(item, False)
Queue.task_done() 在完成一项工作之后，函数向任务已经完成的队列发送一个信号
Queue.join()： 实际上意味着等到队列为空，再执行别的操作 
```

#### 队列示列

**FIFO(先进先出 First In First Out)**  两端开口

Queue类实现了一个基本的先进先出(FIFO)容器，使用put()将元素添加到序列尾端，get()从队列尾部移除元素。

In [4]:
import queue

q = queue.Queue()
for i in range(5):
    q.put(i)
while not q.empty():
    print(q.get()) 

0
1
2
3
4


**LIFO(后进先出 Last In First Out)**  一端开口

与标准FIFO实现Queue不同的是，LifoQueue使用后进先出序（会关联一个栈数据结构）。

In [5]:
import queue

q = queue.LifoQueue()
for i in range(5):
    q.put(i)
while not q.empty():
    print(q.get()) 

4
3
2
1
0


**带优先级的队列**

除了按元素入列顺序外，有时需要根据队列中元素的特性来决定元素的处理顺序。例如，老板的打印任务可能比研发的打印任务优先级更高。PriorityQueue依据队列中内容的排序顺序(sort order)来决定那个元素将被检索。

`__lt__(slef,other)`	 判断self对象是否小于other对象

In [8]:
import queue

class Job(object):
    def __init__(self, priority, description):
        self.priority = priority
        self.description = description
        print('New job:', description)
        return
 
    def __lt__(self, other):
        return self.priority < other.priority


q = queue.PriorityQueue()
q.put( Job(3, 'Mid-level job') )
q.put( Job(10, 'Low-level job') )
q.put( Job(1, 'Important job') ) #数字越小，优先级越高
while not q.empty():
    next_job = q.get() #可根据优先级取序列
    print('Processing job:', next_job.description)

New job: Mid-level job
New job: Low-level job
New job: Important job
Processing job: Important job
Processing job: Mid-level job
Processing job: Low-level job


从上面的执行结果可以看出，优先级值设置越小，越先执行。另外这里是以单线程为例的，在多thread的示例中，多个线程同时get()  item 时，这时就可以根据优先级决定哪一个任务先执行。

#### 队列与线程

在实际使用队列是与线程结合在一起的

In [ ]:
from queue import *
from threading import Thread
import sys

def processor():
    while True:
        if q.empty() == True:
            print('the Queue is empty!')
            sys.exit(1)
        try:
            job = q.get()
            print("I'm operating on job item: {}".format(job))
        except:
            print('Failed to operate on job')

q = Queue()
threads = 4
jobs = ['job1','job2','job3']
for job in jobs:
    print('insert job into the queue: {}'.format(job))
    q.put(job)
    
for i in range(threads):
    th = Thread(target=processor)
    th.setDaemon(True)
    th.start()
    
q.join()

insert job into the queue: job1
insert job into the queue: job2
insert job into the queue: job3
I'm operating on job item: job1
I'm operating on job item: job2
I'm operating on job item: job3
the Queue is empty!
the Queue is empty!
the Queue is empty!
the Queue is empty!


### [python基础之队列](https://www.cnblogs.com/lidagen/p/7252247.html)

queue队列是一只数据结构，数据存放方式类似于列表，但是取数据的方式不同于列表。

队列的数据有三种方式：

　　1、先进先出（FIFO），即哪个数据先存入，取数据的时候先取哪个数据，同生活中的排队买东西
  
 ![queue](queue.png)

　　2、先进后出（LIFO），同栈，即哪个数据最后存入的，取数据的时候先取，同生活中手枪的弹夹，子弹最后放入的先打出

　　3、优先级队列，即存入数据时候加入一个优先级，取数据的时候优先级最高的取出

**先进先出：put存入和get取出**

代码实现 

In [14]:
import queue 
import threading
import time

q = queue.Queue(5) # 加数字限制队列的长度，最多能够存入5个数据，有取出才能继续存入
def put():
    for i in range(10):  #顺序存入数字0到9
        q.put(i)
        time.sleep(1)#延迟存入数字，当队列中没有数据的时候，get函数取数据的时候会阻塞，直到有数据存入后才从阻塞状态释放取出新数据
        
def get():
    for i in range(10):   #从第一个数字0开始取，直到9
        print(q.get())
        
t1 = threading.Thread(target=put, args=())
t2 = threading.Thread(target=get, args=())
t1.start()
t2.start()

0
1
2
3
4
5
6
7
8
9


**先进先出：join阻塞和task_done信号**

代码实现

因为join实际上是一个计数器，put了多少个数据，计数器就是多少，每task_done一次，计数器减1，直到为0才继续执行

In [27]:
import queue 
import threading
import time

q = queue.Queue(5) # 加数字限制队列的长度，最多能够存入5个数据，有取出才能继续存入
def put():
    for i in range(10):  #顺序存入数字0到9
        q.put(i)
    #q.join()    #阻塞进程，直到所有任务完成，取多少次数据task_done多少次才行，否则最后的ok无法打印
    #print('ok')

        
def get():
    for i in range(10):   #从第一个数字0开始取，直到9
        print(q.get())
        q.task_done(  )   # 必须每取走一个数据，发一个task_done信号给join
    #q.task_done()     # 放在这没用，打印出来 OK
        
t1 = threading.Thread(target=put, args=())
t2 = threading.Thread(target=get, args=())
t1.start()
t2.start()

0
1
2
3
4
5
6
7
8
9


**先进后出**

In [22]:
import queue
import threading 
import time

q = queue.LifoQueue()
def put():
    for i in range(10):
        q.put(i)
    q.join()
    print('ok')
    
def get():
    for i in range(10):
        print(q.get())
        q.task_done()
        
t1 = threading.Thread(target=put, args=())
t2 = threading.Thread(target=get, args=())
t1.start()
t2.start()        

9
8
7
6
5
4
3
2
1
0
ok


**按优先级**

不管是数字、字母、列表、元组等（字典、集合没测），使用优先级存数据取数据，队列中的数据必须是同一类型，都是按照实际数据的ascii码表的顺序进行优先级匹配，汉字是按照unicode表（亲测）

In [26]:
import queue

# 列表
l = queue.PriorityQueue()
l.put([1,'aaa'])
l.put([3,'ace'])
l.put([2,333])

while not l.empty():
    print(l.get())
    
# 元祖
y = queue.PriorityQueue()
y.put((1,'aaa'))
y.put((3,'ace'))
y.put((2,333))

while not y.empty():
    print(y.get())
    
# 汉字
h = queue.PriorityQueue()
h.put('我')
h.put('你')
h.put('她')

while not h.empty():
    print(h.get())

[1, 'aaa']
[2, 333]
[3, 'ace']
(1, 'aaa')
(2, 333)
(3, 'ace')
你
她
我


**生产者与消费者模型**

在线程世界里，生产者就是生产数据的线程，消费者就是消费数据的线程。在多线程开发当中，如果生产者处理速度很快，而消费者处理速度很慢，那么生产者就必须等待消费者处理完，才能继续生产数据。同样的道理，如果消费者的处理能力大于生产者，那么消费者就必须等待生产者。为了解决这个问题于是引入了生产者和消费者模式。

生产者消费者模式是通过一个容器来解决生产者和消费者的强耦合问题。生产者和消费者彼此之间不直接通讯，而通过阻塞队列来进行通讯，所以生产者生产完数据之后不用等待消费者处理，直接扔给阻塞队列，消费者不找生产者要数据，而是直接从阻塞队列里取，阻塞队列就相当于一个缓冲区，平衡了生产者和消费者的处理能力。

这就像，在餐厅，厨师做好菜，不需要直接和客户交流，而是交给前台，而客户去饭菜也不需要不找厨师，直接去前台领取即可，这也是一个结耦的过程。

In [38]:
import time,random
import queue,threading

q = queue.Queue()

def Producer(name):
    count = 1
    while count < 10:
        print('making ...')
        q.put(count)
        print('Producer {} has produced {} items...'.format(name,count))
        count += 1
        print('ok...')
        
def Consumer(name):
    count = 1
    while count < 10:
        if not q.empty():
            data = q.get()
            print('Consumer {} has consumed {} items... '.format(name,count))
        else:
            print('--------no item anymore------')
        count += 1
        
p1 = threading.Thread(target=Producer, args=('A',))
c1 = threading.Thread(target=Consumer, args=('B',))
c2 = threading.Thread(target=Consumer, args=('C',))
c3 = threading.Thread(target=Consumer, args=('D',))
p1.start()
c1.start()
c2.start()
c3.start()

making ...--------no item anymore------
Producer A has produced 1 items...
ok...
making ...
Producer A has produced 2 items...

Consumer B has consumed 2 items... 
Consumer B has consumed 3 items... 
--------no item anymore------
--------no item anymore------
--------no item anymore------
--------no item anymore------
--------no item anymore------
--------no item anymore------
ok...
making ...
Producer A has produced 3 items...
ok...
making ...
Producer A has produced 4 items...
ok...
making ...
Producer A has produced 5 items...
ok...
making ...
Producer A has produced 6 items...
ok...
making ...
Producer A has produced 7 items...
ok...
making ...
Producer A has produced 8 items...
ok...
making ...
Producer A has produced 9 items...
ok...
Consumer C has consumed 1 items... Consumer D has consumed 1 items... 
Consumer D has consumed 2 items... 
Consumer D has consumed 3 items... 
Consumer D has consumed 4 items... 
Consumer D has consumed 5 items... 
Consumer D has consumed 6 items... 

### [python基础之Event对象](https://www.cnblogs.com/lidagen/p/7252247.html)

#### Event对象

用于线程间通信，即程序中的其一个线程需要通过判断某个线程的状态来确定自己下一步的操作，就用到了event对象

event对象默认为假（Flase），即遇到event对象在等待就阻塞线程的执行。

event围绕一个标志位在搞。

- event=threading.Event()：条件环境对象，初始值 为False；
- event.isSet()：返回event的状态值；
- event.wait()：如果 event.isSet()==False将阻塞线程；
- event.set()： 设置event的状态值为True，所有阻塞池的线程激活进入就绪状态， 等待操作系统调度；
- event.clear()：恢复event的状态值为False。

Event对象可以用来进行线程通信，**调用event对象的wait方法，线程则会阻塞等待，直到别的线程set之后，才会被唤醒,而 set 线程继续运行。**

![Event](event.png)

##### 主线程和子线程间通信
模拟连接服务器

In [11]:
import threading
import time
event = threading.Event()

def foo():
    print('1: wait server...')
    event.wait() #括号里可以带数字执行，数字表示等待的秒数，不带数字表示一直阻塞状态
    print('4: connect to server')
    
t = threading.Thread(target=foo,args=())  #子线程执行foo函数
t.start()
time.sleep(3)
print('2: start server successful')
time.sleep(3) 
event.set()   #默认为False，set一次表示True，所以子线程里的foo函数解除阻塞状态继续执行
print('3: after set')

1: wait server...
2: start server successful
3: after set4: connect to server



##### 子线程与子线程间通信

In [12]:
import threading
import time
event = threading.Event()

def foo():
    print('1: wait server...')
    event.wait() #括号里可以带数字执行，数字表示等待的秒数，不带数字表示一直阻塞状态
    print('4: connect to server')

def foo1():
    time.sleep(3)
    print('2: start server successful')
    time.sleep(3)
    event.set()   #默认为False，set一次表示True，所以子线程里的foo函数解除阻塞状态继续执行
    print('3: after set')
    
t = threading.Thread(target=foo,args=())  #子线程执行foo函数
t.start()
t2 = threading.Thread(target=foo1,args=())  #子线程执行foo函数
t2.start()

1: wait server...
2: start server successful
3: after set4: connect to server



#####  多线程阻塞

In [16]:
import threading
import time

event = threading.Event()
def foo(i):
    while not event.is_set():  #返回event的状态值，同isSet  
        print('{}: wait server {}...'.format(threading.current_thread().name,i))
        event.wait(2) #等待2秒，如果状态为False，打印一次提示继续等待
    print('{}: connect to server'.format(threading.current_thread().name))
    
for i in range(3): #3个子线程同时等待
    t = threading.Thread(target=foo, args=(i,))
    t.start()
    
print('{}: start server successful'.format(threading.current_thread().name))
time.sleep(10)
print('{}: set to active server'.format(threading.current_thread().name))
event.set()  # 设置标志位为True，event.clear()是恢复event的状态值为False
print('{}: after set'.format(threading.current_thread().name))

Thread-45: wait server 0...Thread-46: wait server 1...

Thread-47: wait server 2...MainThread: start server successful

Thread-45: wait server 0...Thread-46: wait server 1...

Thread-47: wait server 2...
Thread-46: wait server 1...Thread-45: wait server 0...

Thread-47: wait server 2...
Thread-45: wait server 0...Thread-46: wait server 1...

Thread-47: wait server 2...
Thread-45: wait server 0...Thread-46: wait server 1...

Thread-47: wait server 2...
Thread-46: wait server 1...
Thread-45: wait server 0...
MainThread: set to active server
MainThread: after setThread-47: connect to serverThread-46: connect to serverThread-45: connect to server





### 线程通信

In [1]:
import threading
import time

class MyThread(threading.Thread):
    def __init__(self, event):
        super(MyThread, self).__init__()
        self.event = event
 
    def run(self):
        print("thread {} is ready ".format(self.name))
        self.event.wait()
        print("thread {} run".format(self.name))

signal = threading.Event()
 
def main():
    start = time.time()
    for i in range(3):
        t = MyThread(signal)
        t.start()
    time.sleep(3)
    print("after {}s".format(time.time() - start))
    signal.set()
    
main()

thread Thread-6 is ready 
thread Thread-7 is ready 
thread Thread-8 is ready 
after 3.0321013927459717s
thread Thread-7 run
thread Thread-8 runthread Thread-6 run



创建了3个线程，调用线程之后，线程将会被阻塞，sleep 3秒后，才会被唤醒执行

### [ThreadLocal](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431928972981094a382e5584413fa040b46d46cce48e000)

在多线程环境下，每个线程都有自己的数据。一个线程使用自己的局部变量比使用全局变量好，因为局部变量只有线程自己能看见，不会影响其他线程，而全局变量的修改必须加锁。

但是局部变量也有问题，就是在函数调用的时候，传递起来很麻烦：

In [ ]:
def process_student(name):
    std = Student(name)
    # std是局部变量，但是每个函数都要用它，因此必须传进去：
    do_task_1(std)
    do_task_2(std)

def do_task_1(std):
    do_subtask_1(std)
    do_subtask_2(std)

def do_task_2(std):
    do_subtask_2(std)
    do_subtask_2(std)

每个函数一层一层调用都这么传参数那还得了？用全局变量？也不行，因为每个线程处理不同的Student对象，不能共享。

如果用一个全局dict存放所有的Student对象，然后以thread自身作为key获得线程对应的Student对象如何？

In [ ]:
global_dict = {}

def std_thread(name):
    std = Student(name)
    # 把std放到全局变量global_dict中：
    global_dict[threading.current_thread()] = std
    do_task_1()
    do_task_2()

def do_task_1():
    # 不传入std，而是根据当前线程查找：
    std = global_dict[threading.current_thread()]

def do_task_2():
    # 任何函数都可以查找出当前线程的std变量：
    std = global_dict[threading.current_thread()]

这种方式理论上是可行的，它最大的优点是消除了std对象在每层函数中的传递问题，但是，每个函数获取std的代码有点丑。

有没有更简单的方式？

ThreadLocal应运而生，不用查找dict，ThreadLocal帮你自动做这件事：

In [12]:
import threading

# 创建全局ThreadLocal对象:
local_school = threading.local()

def process_student():
    # 获取当前线程关联的student:
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))

def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()

t1 = threading.Thread(target= process_thread, args=('Alice',), name='Thread-A')
t2 = threading.Thread(target= process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)


全局变量local_school就是一个ThreadLocal对象，每个Thread对它都可以读写student属性，但互不影响。你可以把local_school看成全局变量，但每个属性如local_school.student都是线程的局部变量，可以任意读写而互不干扰，也不用管理锁的问题，ThreadLocal内部会处理。

可以理解为全局变量local_school是一个dict，不但可以用local_school.student，还可以绑定其他变量，如local_school.teacher等等。

ThreadLocal最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

**小结**

一个ThreadLocal变量虽然是全局变量，但每个线程都只能读写自己线程的独立副本，互不干扰。ThreadLocal解决了参数在一个线程中各个函数之间互相传递的问题。

### [深入理解Python中的ThreadLocal变量（上）](http://python.jobbole.com/86150/)

我们知道多线程环境下，每一个线程均可以使用所属进程的全局变量。如果一个线程对全局变量进行了修改，将会影响到其他所有的线程。为了避免多个线程同时对变量进行修改，引入了线程同步机制，通过互斥锁，条件变量或者读写锁来控制对全局变量的访问。

只用全局变量并不能满足多线程环境的需求，很多时候线程还需要拥有自己的私有数据，这些数据对于其他线程来说不可见。因此线程中也可以使用局部变量，局部变量只有线程自身可以访问，同一个进程下的其他线程不可访问。

有时候使用局部变量不太方便，因此 python 还提供了 ThreadLocal 变量，它本身是一个全局变量，但是每个线程却可以利用它来保存属于自己的私有数据，这些私有数据对其他线程也是不可见的。下图给出了线程中这几种变量的存在情况：

![threadlocal](threadlocal_1.png)

#### 全局 VS 局部变量

首先借助一个小程序来看看多线程环境下全局变量的同步问题。

In [3]:
import threading
global_num = 0
 
def thread_cal():
    global global_num
    for i in range(100000):
        global_num += 1

# Get 10 threads, run them and wait them all finished.
threads = []
for i in range(10):
    threads.append(threading.Thread(target=thread_cal))
    threads[i].start()
for i in range(10):
    threads[i].join()

# Value of global variable can be confused.
print(global_num)

690856


这里我们创建了10个线程，每个线程均对全局变量 global_num 进行1000000次的加1操作（循环1000次加1是为了延长单个线程执行时间，使线程执行时被中断切换），当10个线程执行完毕时，全局变量的值是多少呢？答案是不确定。**简单来说是因为 global_num += 1 并不是一个原子操作，因此执行过程可能被其他线程中断，导致其他线程读到一个脏值**。以两个线程执行 +1 为例，其中一个可能的执行序列如下（此情况下最后结果为1）

![threadlocal](threadlocal_2.png)

多线程中使用**全局变量**时普遍存在这个问题，解决办法也很简单，可以使用互斥锁、条件变量或者是读写锁。下面考虑用互斥锁来解决上面代码的问题，只需要在进行 +1 运算前加锁，运算完毕释放锁即可，这样就可以保证运算的原子性。

In [4]:
import threading
global_num = 0
l = threading.Lock()


def thread_cal():
    global global_num
    for i in range(100000):
        l.acquire()
        global_num += 1
        l.release()
        
# Get 10 threads, run them and wait them all finished.
threads = []
for i in range(10):
    threads.append(threading.Thread(target=thread_cal))
    threads[i].start()
for i in range(10):
    threads[i].join()

# Value of global variable can be confused.
print(global_num)

1000000


在线程中使用**局部变量**则不存在这个问题，因为每个线程的局部变量不能被其他线程访问。下面我们用10个线程分别对各自的局部变量进行100000次加1操作，每个线程结束时打印一共执行的操作次数

In [5]:
def show(num):
    print(threading.current_thread().getName(), num)

def thread_cal():
    local_num = 0
    for i in range(1000):
        local_num += 1
    show(local_num)

threads = []
for i in range(10):
    threads.append(threading.Thread(target=thread_cal))
    threads[i].start()

Thread-46 1000
Thread-47Thread-48  1000
1000
Thread-49 1000
Thread-50Thread-51 1000
Thread-52 1000
Thread-53Thread-54 1000 Thread-55
1000 1000
 1000



#### Thread-local 对象

上面程序中我们需要给 show 函数传递 local_num 局部变量，并没有什么不妥。不过考虑在实际生产环境中，我们可能会调用很多函数，每个函数都需要很多局部变量，这时候用传递参数的方法会很不友好。

为了解决这个问题，一个直观的的方法就是建立一个全局字典，保存进程 ID 到该进程局部变量的映射关系，运行中的线程可以根据自己的 ID 来获取本身拥有的数据。这样，就可以避免在函数调用中传递参数，如下示例：

In [7]:
global_data = {}

def show():
    cur_thread = threading.current_thread()
    print(cur_thread.getName(),global_data[cur_thread])
    
def thread_cal():
    global global_data
    cur_thread = threading.current_thread()
    global_data[cur_thread] = 0
    for i in range(1000):
        global_data[cur_thread] += 1
    show()
    
threads = []
for i in range(10):
    threads.append(threading.Thread(target=thread_cal))
    threads[i].start() 

Thread-56 1000
Thread-57 1000Thread-58
 1000Thread-59
 1000
Thread-60 1000
Thread-61 1000
Thread-62 1000
Thread-63 1000
Thread-64 1000
Thread-65 1000


保存一个全局字典，然后将线程标识符作为key，相应线程的局部数据作为 value，这种做法并不完美。首先，每个函数在需要线程局部数据时，都需要先取得自己的线程ID，略显繁琐。更糟糕的是，这里并没有真正做到线程之间数据的隔离，因为每个线程都可以读取到全局的字典，每个线程都可以对字典内容进行更改。

为了更好解决这个问题，python 线程库实现了 ThreadLocal 变量（很多语言都有类似的实现，比如Java）。ThreadLocal 真正做到了线程之间的数据隔离，并且使用时不需要手动获取自己的线程 ID，如下示例：

In [11]:
global_data = threading.local()

def show():
    cur_thread = threading.current_thread()
    print(cur_thread.getName(),global_data.num)
    
def thread_cal():
    global_data.num = 0
    for i in range(1000):
        global_data.num += 1
    show()
    
threads = []
for i in range(10):
    threads.append(threading.Thread(target=thread_cal))
    threads[i].start()
    
print('Main thread: ',global_data.__dict__)

Thread-96 Thread-971000 
1000
Thread-98 1000
Thread-99Thread-100 1000
 1000
Thread-101 1000Thread-102 1000

Thread-103 1000
Thread-104 1000
Thread-105Main thread:  {}
 1000


## [线程中的信息隔离](https://zhuanlan.zhihu.com/p/36850533)

什么是信息隔离？

比如说，咱有两个线程，线程A里的变量，和线程B里的变量值不能共享。这就是信息隔离。

那么，如何实现信息隔离呢？

在Python中，其提供了threading.local这个类，可以很方便的控制变量的隔离，即使是同一个变量，在不同的线程中，其值也是不能共享的。

In [13]:
from threading import local, Thread, currentThread

# 定义一个local实例
local_data = local()
# 在主线中，存入name这个变量
local_data.name = 'local_data'


class MyThread(Thread):
    def run(self):
        print("赋值前-子线程：", currentThread(),local_data.__dict__)
        # 在子线程中存入name这个变量
        local_data.name = self.getName()
        print("赋值后-子线程：",currentThread(), local_data.__dict__)


if __name__ == '__main__':
    print("开始前-主线程：",local_data.__dict__)

    t1 = MyThread()
    t1.start()
    t1.join()

    t2 = MyThread()
    t2.start()
    t2.join()

    print("结束后-主线程：",local_data.__dict__)

开始前-主线程： {'name': 'local_data'}
赋值前-子线程： <MyThread(Thread-106, started 67256)> {}
赋值后-子线程： <MyThread(Thread-106, started 67256)> {'name': 'Thread-106'}
赋值前-子线程： <MyThread(Thread-107, started 24188)> {}
赋值后-子线程： <MyThread(Thread-107, started 24188)> {'name': 'Thread-107'}
结束后-主线程： {'name': 'local_data'}


从输出来看，我们可以知道，local实际是一个字典型的对象，其内部可以以key-value的形式存入你要做信息隔离的变量。local实例可以是全局唯一的，只有一个。因为你在给local存入或访问变量时，它会根据当前的线程的不同从不同的存储空间存入或获取。

基于此，我们可以得出以下三点结论：

- 主线程中的变量，不会因为其是全局变量，而被子线程获取到；
- 主线程也不能获取到子线程中的变量；
- 子线程与子线程之间的变量也不能互相访问。

所以如果想在当前线程保存一个全局值，并且各自线程（包括主线程）互不干扰，使用local类吧。


## [如何创建线程池](https://zhuanlan.zhihu.com/p/36948443)

在使用多线程处理任务时也不是线程越多越好，由于在切换线程的时候，需要切换上下文环境，依然会造成cpu的大量开销。为解决这个问题，线程池的概念被提出来了。预先创建好一个较为优化的数量的线程，让过来的任务立刻能够使用，就形成了线程池。

在Python3中，创建线程池是通过concurrent.futures函数库中的ThreadPoolExecutor类来实现的。

In [1]:
import time, threading
from concurrent.futures import ThreadPoolExecutor

def target():
    for i in range(5):
        print('running thread-{}:{}'.format(threading.get_ident(),i))
        time.sleep(1)

# 设置线程池最大线程数
pool = ThreadPoolExecutor(5)

for i in range(100):
    pool.submit(target) # 往线程中提交，并运行

running thread-2560:0
running thread-8208:0
running thread-17804:0
running thread-18000:0running thread-18264:0

running thread-2560:1
running thread-8208:1
running thread-17804:1
running thread-18264:1
running thread-18000:1
running thread-2560:2
running thread-8208:2
running thread-17804:2
running thread-18264:2
running thread-18000:2
running thread-2560:3
running thread-8208:3
running thread-17804:3
running thread-18264:3
running thread-18000:3
running thread-2560:4
running thread-8208:4
running thread-17804:4
running thread-18264:4
running thread-18000:4
running thread-2560:0
running thread-8208:0
running thread-17804:0
running thread-18264:0
running thread-18000:0
running thread-2560:1
running thread-8208:1
running thread-17804:1
running thread-18264:1
running thread-18000:1
running thread-2560:2
running thread-8208:2
running thread-17804:2
running thread-18264:2
running thread-18000:2
running thread-2560:3
running thread-8208:3
running thread-17804:3
running thread-18264:3
runnin

running thread-17804:3
running thread-8208:3
running thread-2560:3
running thread-18264:3
running thread-18000:3
running thread-17804:4
running thread-8208:4running thread-2560:4

running thread-18264:4
running thread-18000:4
running thread-17804:0
running thread-8208:0running thread-2560:0

running thread-18264:0
running thread-18000:0
running thread-17804:1
running thread-8208:1running thread-2560:1

running thread-18264:1
running thread-18000:1
running thread-17804:2
running thread-8208:2running thread-2560:2

running thread-18264:2
running thread-18000:2
running thread-17804:3
running thread-8208:3running thread-2560:3

running thread-18264:3
running thread-18000:3
running thread-17804:4
running thread-2560:4running thread-8208:4

running thread-18264:4
running thread-18000:4
running thread-17804:0
running thread-2560:0running thread-8208:0

running thread-18264:0
running thread-18000:0
running thread-17804:1
running thread-8208:1
running thread-2560:1
running thread-18264:1
runnin

## [Python 线程池原理及实现](https://www.jianshu.com/p/afd9b3deb027)

### 概述

传统多线程方案会使用“**即时创建， 即时销毁**”的策略。尽管与创建进程相比，创建线程的时间已经大大的缩短，但是如果提交给线程的任务是执行时间较短，而且执行次数极其频繁，那么服务器将处于不停的创建线程，销毁线程的状态。

一个线程的运行时间可以分为3部分：

1. 线程的启动时间
2. 线程体的运行时间
3. 线程的销毁时间

在多线程处理的情景中，**如果线程不能被重用，就意味着每次创建都需要经过启动、销毁和运行3个过程**。这必然会增加系统相应的时间，降低了效率。

使用线程池：(线程重用,避免多次创建和销毁线程)

由于线程预先被创建并放入线程池中，同时处理完当前任务之后并不销毁而是被安排处理下一个任务，因此能够避免多次创建线程，从而节省线程创建和销毁的开销，能带来更好的性能和系统稳定性。

![thread_pool](thread_pool.png)

### 线程池模型


In [9]:
import threading
from queue import Queue
import time

# 创建队列实例， 用于存储任务 
queue = Queue()

# 定义需要线程池执行的任务
def do_job():
    while True:
        i = queue.get() # queue内部实现了相关的锁，如果queue的为空，则get元素的时候会被阻塞，知道队列里面被其他线程写入数据。
        time.sleep(1)
        print('indx {}, current:{}'.format(i, threading.current_thread()))
        queue.task_done() # 因为join实际上是一个计数器，put了多少个数据，计数器就是多少，每task_done一次，计数器减1，直到为0才继续执行
        
if __name__ == '__main__':
    # 创建包括3个线程的线程池
    for i in range(3):
        t = threading.Thread(target=do_job)
        t.daemon = True
        t.start()
    print('main thread')
    # 模拟创建线程池3秒后塞进10个任务到队列
    time.sleep(3) 
    print('put in data')
    for i in range(10):
        queue.put(i) # 当写入数据的时候，如果元素个数大于队列的长度，也会被阻塞。也就是在 put 或 get的时候都会获得Lock。
    print('put complete')    
    queue.join() # 阻塞进程，直到所有任务完成，取多少次数据task_done多少次才行，否则最后的finish无法打印
    print('finish')

main thread
put in data
put complete
indx 2, current:<Thread(Thread-14, started daemon 17872)>indx 1, current:<Thread(Thread-13, started daemon 21936)>indx 0, current:<Thread(Thread-12, started daemon 19388)>


indx 4, current:<Thread(Thread-12, started daemon 19388)>indx 3, current:<Thread(Thread-13, started daemon 21936)>

indx 5, current:<Thread(Thread-14, started daemon 17872)>
indx 7, current:<Thread(Thread-12, started daemon 19388)>indx 6, current:<Thread(Thread-13, started daemon 21936)>

indx 8, current:<Thread(Thread-14, started daemon 17872)>
indx 9, current:<Thread(Thread-12, started daemon 19388)>
finish


可以看到所有任务都是在这几个线程中完成Thread-(12-14)


### 线程池原理


线程池基本原理： 我们把任务放进队列中去，然后开N个线程，每个线程都去队列中取一个任务，执行完了之后告诉系统说我执行完了，然后接着去队列中取下一个任务，直至队列中所有任务取空，退出线程。

上面这个例子生成一个有3个线程的线程池，每个线程都无限循环阻塞读取Queue队列的任务,所有任务都只会让这3个预生成的线程来处理。

具体工作描述如下：

1. 创建Queue.Queue()实例，然后对它填充数据或任务
2. 生成守护线程池，把线程设置成了daemon守护线程
3. 每个线程无限循环阻塞读取queue队列的项目item，并处理
4. 每次完成一次工作后，使用queue.task_done()函数向任务已经完成的队列发送一个信号
5. 主线程设置queue.join()阻塞，直到任务队列已经清空了，解除阻塞，向下执行

这个模式下有几个注意的点：

- 将线程池的线程设置成daemon守护线程，意味着主线程退出时，守护线程也会自动退出，如果使用默认 daemon=False的话， 非daemon的线程会阻塞主线程的退出，所以即使queue队列的任务已经完成线程池依然阻塞无限循环等待任务，使得主线程也不会退出。
- 当主线程使用了queue.join()的时候，说明主线程会阻塞直到queue已经是清空的，而主线程怎么知道queue已经是清空的呢？就是通过每次线程queue.get()后并处理任务后，发送queue.task_done()信号，queue的数据就会减1，直到queue的数据是空的，queue.join()解除阻塞，向下执行。
- 这个模式主要是以队列queue的任务来做主导的，做完任务就退出，由于线程池是daemon的，所以主退出线程池所有线程都会退出。 有别于我们平时可能以队列主导thread.join()阻塞，这种线程完成之前阻塞主线程。看需求使用哪个join()：
   - 如果是想做完一定数量任务的队列就结束，使用queue.join()，比如爬取指定数量的网页；
   - 如果是想线程做完任务就结束，使用thread.join()
